In [5]:
# === Phase 0: Environment Setup (CRITICAL - Must restart after this) ===
# Run this entire cell, then restart the kernel before running any other code

# Install core ML libraries (NumPy <2.0 for morl-baselines compatibility)
!pip install "numpy<2.0,>=1.26.4" scipy==1.11.4

# Install RL stack (pycddlib-standalone will be installed but needs patch)
!pip install mo-gymnasium morl-baselines gymnasium torch pycddlib-standalone

# Install data science libraries
!pip install pandas matplotlib seaborn tqdm

print("✅ Installation complete!")
print("🔄 CRITICAL: Restart your notebook kernel NOW (Runtime → Restart runtime)")
print("After restart, run Phases 1-6, then the monkey-patch below, then Phase 7.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 97.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompati

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.6/479.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.8/303.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.6 MB/s eta 0:00:00
  Created wheel for pycddlib-standalone: filename=pycddlib_standalone-3.0.0-cp312-cp312-linux_x86_64.whl size=635355 sha256=173fa2a6cee48a0328ff00cd3617afd002d69563a10ea7c53bea16649d6504de
  Stored in directory: /root/.cache/pi

Phase 1: Data Loading & Validation
Goal: Load MIMIC-III, SIDER, and DDInter files; perform initial validations to avoid propagating errors downstream

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# === Phase 1: Data Loading & Validation ===
# Load all datasets from your Google Drive path

# MIMIC-III Core Files
prescriptions = pd.read_csv('/content/drive/MyDrive/ML Patent/polypharmacy_project/data/mimic3/PRESCRIPTIONS.csv')
admissions = pd.read_csv('/content/drive/MyDrive/ML Patent/polypharmacy_project/data/mimic3/ADMISSIONS.csv')
patients = pd.read_csv('/content/drive/MyDrive/ML Patent/polypharmacy_project/data/mimic3/PATIENTS.csv')

# DDInter Drug-Drug Interaction Database
ddinter = pd.read_csv('/content/drive/MyDrive/ML Patent/polypharmacy_project/data/ddinter/ddinter_downloads_code_L.csv')

# SIDER Side Effect Database
drug_names = pd.read_csv('/content/drive/MyDrive/ML Patent/polypharmacy_project/data/sider/drug_names.tsv',
                         sep='\t', header=None, names=['stitch_id_flat','stitch_id_stereo','drug_name'])
drug_atc = pd.read_csv('/content/drive/MyDrive/ML Patent/polypharmacy_project/data/sider/drug_atc.tsv',
                       sep='\t', header=None, names=['stitch_id_flat','atc_code'])
side_effects = pd.read_csv('/content/drive/MyDrive/ML Patent/polypharmacy_project/data/sider/meddra_all_se.tsv.gz',
                           sep='\t', header=None, names=['stitch_id_flat','stitch_id_stereo','umls_cui','meddra_type','umls_label','meddra_code'])

# === Validation Checks ===
print("=== Data Loading Summary ===")
print(f"PRESCRIPTIONS: {prescriptions.shape}")
print(f"ADMISSIONS: {admissions.shape}")
print(f"PATIENTS: {patients.shape}")
print(f"DDInter: {ddinter.shape}")
print(f"SIDER drug_names: {drug_names.shape}")
print(f"SIDER drug_atc: {drug_atc.shape}")
print(f"SIDER side_effects: {side_effects.shape}")

# Check key columns exist
print("\n=== Key Columns Check ===")
print(f"PRESCRIPTIONS columns: {list(prescriptions.columns)[:8]}...")
print(f"DDInter columns: {list(ddinter.columns)}")
print(f"drug_names columns: {list(drug_names.columns)}")

# Preview first few rows to verify data integrity
print("\n=== Data Preview ===")
print("PRESCRIPTIONS sample:")
print(prescriptions.head(3))
print("\nDDInter sample:")
print(ddinter.head(3))

print("\n✅ Phase 1 Complete: All data loaded successfully!")


=== Data Loading Summary ===
PRESCRIPTIONS: (10398, 19)
ADMISSIONS: (129, 19)
PATIENTS: (100, 8)
DDInter: (65389, 5)
SIDER drug_names: (1430, 3)
SIDER drug_atc: (1560, 2)
SIDER side_effects: (309849, 6)

=== Key Columns Check ===
PRESCRIPTIONS columns: ['row_id', 'subject_id', 'hadm_id', 'icustay_id', 'startdate', 'enddate', 'drug_type', 'drug']...
DDInter columns: ['DDInterID_A', 'Drug_A', 'DDInterID_B', 'Drug_B', 'Level']
drug_names columns: ['stitch_id_flat', 'stitch_id_stereo', 'drug_name']

=== Data Preview ===
PRESCRIPTIONS sample:
   row_id  subject_id  hadm_id  icustay_id            startdate  \
0   32600       42458   159647         NaN  2146-07-21 00:00:00   
1   32601       42458   159647         NaN  2146-07-21 00:00:00   
2   32602       42458   159647         NaN  2146-07-21 00:00:00   

               enddate drug_type                         drug  \
0  2146-07-22 00:00:00      MAIN  Pneumococcal Vac Polyvalent   
1  2146-07-22 00:00:00      MAIN                    Bisac

Phase 2: Drug Normalization & ATC Mapping
Goal: Standardize drug names and align MIMIC-III drugs with SIDER's ATC codes using fuzzy matching and manual curation.

In [6]:
from difflib import get_close_matches
import pandas as pd
import numpy as np
from tqdm import tqdm
import os # Import the os module

# Define the output path early
out_path = '/content/drive/MyDrive/ML Patent/polypharmacy_project/drug_to_atc_mapping.csv'

# Check if the mapping file already exists
if os.path.exists(out_path):
    drug_mapping = pd.read_csv(out_path)
    print(f"✅ Drug mapping already exists at {out_path}. Loaded {len(drug_mapping)} entries.")
    print(f"Total drugs: {len(drug_mapping)}")
    print(f"Matched to ATC: {drug_mapping['atc_code'].notna().sum()} ({100 * drug_mapping['atc_code'].notna().mean():.1f}%)")
else:
    print("Drug mapping file not found. Proceeding with mapping process...")
    # 1. Drug Normalization
    def normalize_drug_name(drug_string):
        """Normalize drug names by removing formulations and standardizing case."""
        if pd.isna(drug_string):
            return ""
        drug = str(drug_string).lower().strip()
        # Remove content in parentheses and brackets
        drug = drug.split('(')[0].split('[')[0].strip()
        # Remove common dosage forms and routes
        suffixes = [' tab', ' tablet', ' cap', ' capsule', ' injection', ' inj', ' iv',
                    ' oral', ' susp', ' suspension', ' solution', ' liquid', ' cream',
                    ' ointment', ' gel', ' drops', ' spray', ' powder', ' patch']
        for suffix in suffixes:
            drug = drug.replace(suffix, '')
        return drug.strip()

    # Apply to all unique drugs in prescriptions
    mimic_drugs_clean = pd.DataFrame({
        'original_name': prescriptions['drug'].unique(),
        'normalized_name': [normalize_drug_name(d) for d in prescriptions['drug'].unique()]
    })
    print(f"Total unique drugs to map: {len(mimic_drugs_clean)}")

    # 2. SIDER Preparation
    # Clean SIDER drug names and create lookup dictionary
    drug_names['drug_name_clean'] = drug_names['drug_name'].astype(str).str.lower().str.strip()
    sider_drug_atc = drug_names.merge(drug_atc, on='stitch_id_flat', how='inner')

    # Create lookup: normalized drug name -> ATC code
    sider_lookup = {}
    for _, row in sider_drug_atc.iterrows():
        if pd.notna(row['drug_name_clean']) and pd.notna(row['atc_code']):
            sider_lookup[row['drug_name_clean']] = row['atc_code']

    print(f"SIDER lookup dictionary size: {len(sider_lookup)}")

    # 3. Fuzzy Matching Function
    def find_best_match(mimic_drug, sider_lookup, threshold=0.75):
        """Find best SIDER match using exact then fuzzy matching."""
        if not mimic_drug:
            return None, None, 0.0

        # Exact match
        if mimic_drug in sider_lookup:
            return mimic_drug, sider_lookup[mimic_drug], 1.0

        # Fuzzy match
        matches = get_close_matches(mimic_drug, list(sider_lookup.keys()), n=1, cutoff=threshold)
        if matches:
            return matches[0], sider_lookup[matches[0]], threshold

        return None, None, 0.0

    # 4. Batch Matching with Progress Bar
    results = []
    for _, row in tqdm(mimic_drugs_clean.iterrows(), total=len(mimic_drugs_clean), desc="Mapping drugs"):
        norm = row['normalized_name']
        match, atc, conf = find_best_match(norm, sider_lookup)
        results.append({
            'original_name': row['original_name'],
            'normalized_name': norm,
            'sider_match': match,
            'atc_code': atc,
            'match_confidence': conf
        })

    drug_mapping = pd.DataFrame(results)
    print(f"Successfully mapped: {drug_mapping['atc_code'].notna().sum()}/{len(drug_mapping)}")

    # 5. Manual Mappings for Common Missed Drugs
    manual_mappings = {
        'heparin': 'B01AB01',
        'insulin': 'A10AB01',
        'aspirin': 'B01AC06',
        'paracetamol': 'N02BE01',
        'acetaminophen': 'N02BE01',
        'morphine': 'N02AA01',
        'fentanyl': 'N02AB03',
        'midazolam': 'N05CD08',
        'propofol': 'N01AX10',
        'norepinephrine': 'C01CA03',
        'dopamine': 'C01CA04',
        'vancomycin': 'J01XA01',
        'cefazolin': 'J01DB04',
        'ceftriaxone': 'J01DD04',
        'meropenem': 'J01DH02',
        'piperacillin': 'J01CA12',
        'tazobactam': 'J01CG02',
        'metronidazole': 'P01AB01',
        'fluconazole': 'J02AC01',
        'furosemide': 'C03CA01',
        'metoprolol': 'C07AB02'
    }

    # Apply manual mappings only to unmatched drugs
    for idx, row in drug_mapping.iterrows():
        if pd.isna(row['atc_code']):
            for key, atc in manual_mappings.items():
                if key in row['normalized_name']:
                    drug_mapping.at[idx, 'atc_code'] = atc
                    drug_mapping.at[idx, 'sider_match'] = key
                    drug_mapping.at[idx, 'match_confidence'] = 0.95
                    break

    # 6. Final Statistics and Save
    print("\n=== Final Mapping Statistics ===")
    print(f"Total drugs: {len(drug_mapping)}")
    print(f"Auto-matched: {(drug_mapping['match_confidence'] >= 0.75).sum()}")
    print(f"Manual-mapped: {(drug_mapping['match_confidence'] == 0.95).sum()}")
    print(f"Unmatched: {drug_mapping['atc_code'].isna().sum()}")

    # Save mapping
    drug_mapping.to_csv(out_path, index=False)
    print(f"\n✅ Saved mapping to: {out_path}")

    # Preview unmatched drugs for manual review
    unmatched = drug_mapping[drug_mapping['atc_code'].isna()]['normalized_name'].tolist()
    if unmatched:
        print(f"\nUnmatched drugs (first 10): {unmatched[:10]}")


✅ Drug mapping already exists at /content/drive/MyDrive/ML Patent/polypharmacy_project/drug_to_atc_mapping.csv. Loaded 592 entries.
Total drugs: 592
Matched to ATC: 60 (10.1%)


Phase 3: Cohort Extraction (Adult Polypharmacy)
Goal: Filter for adult admissions and identify polypharmacy episodes (≥5 meds on the same day).

In [8]:
import numpy as np
admissions['admittime'] = pd.to_datetime(admissions['admittime'], errors='coerce')
patients['dob'] = pd.to_datetime(patients['dob'], errors='coerce')
adm_demo = admissions.merge(patients, on='subject_id', how='left')
adm_demo['age'] = (adm_demo['admittime'].dt.year - adm_demo['dob'].dt.year)
adm_demo.loc[adm_demo['age'] < 0, 'age'] = np.nan
adm_demo.loc[adm_demo['age'] > 120, 'age'] = 90
adm_demo = adm_demo[adm_demo['age'] >= 18]

# Merge to Rx and get normalized ATC
prescriptions['startdate'] = pd.to_datetime(prescriptions['startdate'], errors='coerce')
adult_cohort = prescriptions.merge(adm_demo[['subject_id','hadm_id','age']], on=['subject_id','hadm_id'], how='inner')
drug_mapping = pd.read_csv('/content/drive/MyDrive/ML Patent/polypharmacy_project/drug_to_atc_mapping.csv')
adult_cohort = adult_cohort.merge(drug_mapping[['original_name','atc_code']], left_on='drug', right_on='original_name', how='left')
adult_cohort['start_day'] = adult_cohort['startdate'].dt.date
poly_days = (adult_cohort.groupby(['hadm_id','start_day']).agg({'drug':'nunique'}).reset_index().rename(columns={'drug':'num_drugs'}))
poly_adm = poly_days[poly_days['num_drugs'] >= 5]['hadm_id'].unique()
poly_cohort = adult_cohort[adult_cohort['hadm_id'].isin(poly_adm)]

# Save for next step
poly_cohort.to_csv('/content/drive/MyDrive/ML Patent/polypharmacy_project/polypharmacy_cohort.csv', index=False)


Phase 4: Feature Engineering (Drug-Drug Interactions & Tolerability)
Goal: Compute DDI statistics and aggregate side effect burden for each admission.

In [9]:
from itertools import combinations
from tqdm import tqdm
import os

# --- DDI Analysis ---
poly_cohort['drug_norm'] = poly_cohort['drug'].str.lower().str.strip()
ddinter = ddinter.rename(columns={'Level':'severity'}) if 'Level' in ddinter.columns else ddinter

output_ddi_path = '/content/drive/MyDrive/ML Patent/polypharmacy_project/polypharmacy_cohort_with_ddi.csv'

if os.path.exists(output_ddi_path):
    print(f"✅ DDI features already exist at {output_ddi_path}. Loading existing file.")
    poly_cohort = pd.read_csv(output_ddi_path)
else:
    print("DDI features file not found. Calculating DDI features...")
    ddi_results = []
    for hadm_id, group in tqdm(poly_cohort.groupby('hadm_id')):
        drugs = group['drug_norm'].unique()
        pairs = combinations(drugs, 2)
        ddi_count = major_count = moderate_count = 0
        for d1, d2 in pairs:
            match = ddinter[( (ddinter['Drug_A'].str.lower() == d1) & (ddinter['Drug_B'].str.lower() == d2)) |
                            (ddinter['Drug_A'].str.lower() == d2) & (ddinter['Drug_B'].str.lower() == d1)]
            if not match.empty:
                ddi_count += 1
                levels = match['severity'].unique()
                if any(l.lower() == 'major' for l in levels): major_count += 1
                if any(l.lower() == 'moderate' for l in levels): moderate_count += 1
        ddi_results.append({'hadm_id': hadm_id, 'ddi_count': ddi_count, 'major_count': major_count, 'moderate_count': moderate_count, 'n_drugs': len(drugs)})
    ddi_df = pd.DataFrame(ddi_results)

    # Merge DDI features back to poly_cohort
    poly_cohort = poly_cohort.merge(ddi_df, on='hadm_id', how='left')
    poly_cohort.to_csv(output_ddi_path, index=False)
    print(f"✅ DDI features saved to: {output_ddi_path}")

# --- Tolerability (Side Effects) ---
output_tol_path = '/content/drive/MyDrive/ML Patent/polypharmacy_project/polypharmacy_tolerability.csv'

if os.path.exists(output_tol_path):
    print(f"✅ Tolerability features already exist at {output_tol_path}. Loading existing file.")
    tol_features = pd.read_csv(output_tol_path)
else:
    print("Tolerability features file not found. Calculating tolerability features...")

    # Re-merge drug_mapping to ensure 'atc_code' is available for side effect calculation
    # This is crucial if poly_cohort was loaded from polypharmacy_cohort_with_ddi.csv which might not retain 'atc_code'
    if 'atc_code' not in poly_cohort.columns:
        print("Re-merging drug_mapping to poly_cohort for side effect calculation (atc_code missing).")
        # Assuming 'drug' column in poly_cohort corresponds to 'original_name' in drug_mapping
        drug_mapping_for_se = drug_mapping[['original_name', 'atc_code']].drop_duplicates()
        # Drop any existing 'original_name' column that might conflict before merging
        poly_cohort = poly_cohort.drop(columns=['original_name'], errors='ignore')
        poly_cohort = poly_cohort.merge(drug_mapping_for_se, left_on='drug', right_on='original_name', how='left')

    sider_drug_atc_map = drug_names.merge(drug_atc, on='stitch_id_flat', how='inner')
    se_atc = sider_drug_atc_map.merge(side_effects, on='stitch_id_flat', how='inner')
    se_counts = se_atc.groupby('atc_code')['umls_cui'].nunique().reset_index()
    se_counts.columns = ['atc_code','side_effect_count']

    # Merge side effect counts to the cohort
    poly_cohort_with_se = poly_cohort.merge(se_counts, on='atc_code', how='left')

    # Aggregate tolerability for each admission
    tol_features = poly_cohort_with_se.groupby('hadm_id').agg({
        'side_effect_count':['sum','mean'],
        'drug':'nunique',
        'ddi_count':'first',
        'major_count':'first',
        'moderate_count':'first'
    }).reset_index()
    tol_features.columns = ['hadm_id', 'total_se', 'mean_se', 'n_drugs', 'ddi_count', 'major_count', 'moderate_count']

    tol_features.to_csv(output_tol_path, index=False)
    print(f"✅ Tolerability features saved to: {output_tol_path}")


✅ DDI features already exist at /content/drive/MyDrive/ML Patent/polypharmacy_project/polypharmacy_cohort_with_ddi.csv. Loading existing file.
✅ Tolerability features already exist at /content/drive/MyDrive/ML Patent/polypharmacy_project/polypharmacy_tolerability.csv. Loading existing file.


Phase 5: Modeling Table Preparation
Goal: Join all structured features, resolve missing data, and split into train/test for RL.

In [10]:
tol = pd.read_csv('/content/drive/MyDrive/ML Patent/polypharmacy_project/polypharmacy_tolerability.csv')
admissions = pd.read_csv('/content/drive/MyDrive/ML Patent/polypharmacy_project/data/mimic3/ADMISSIONS.csv')
patients = pd.read_csv('/content/drive/MyDrive/ML Patent/polypharmacy_project/data/mimic3/PATIENTS.csv')

admissions['admittime'] = pd.to_datetime(admissions['admittime'], errors='coerce')
patients['dob'] = pd.to_datetime(patients['dob'], errors='coerce')
adm_demo = admissions.merge(patients, on='subject_id', how='left')
adm_demo['age'] = adm_demo['admittime'].dt.year - adm_demo['dob'].dt.year
adm_demo.loc[adm_demo['age'] < 0, 'age'] = np.nan
adm_demo.loc[adm_demo['age'] > 120, 'age'] = 90
admissions_min = admissions[['subject_id','hadm_id']].drop_duplicates()
model = tol.merge(admissions_min, on='hadm_id', how='left')
age_table = adm_demo[['subject_id','hadm_id','age']].dropna().drop_duplicates()
model = model.merge(age_table, on=['subject_id','hadm_id'], how='left')

for c in ['age','n_drugs','total_se','ddi_count']:
    model[c] = model[c].fillna(model[c].median())
model_path = '/content/drive/MyDrive/ML Patent/polypharmacy_project/polypharmacy_model_table.csv'
model.to_csv(model_path, index=False)


Phase 6: RL Environment & Agent (Single-Step Prototype)
Goal: Build a Gymnasium environment reflecting your state/action/reward logic and train a multi-objective RL agent.


In [11]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.envs.registration import EnvSpec  # CRITICAL IMPORT
import numpy as np
import pandas as pd

class PolypharmacyEnv(gym.Env):
    def __init__(self, df: pd.DataFrame):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.n = len(self.df)

        # State: [age, n_drugs, total_se, ddi_count]
        self.observation_space = spaces.Box(
            low=0.0, high=1.0, shape=(4,), dtype=np.float32
        )

        # Actions: 0=reduce risk, 1=maintain
        self.action_space = spaces.Discrete(2)

        # CRITICAL: Define reward_space for morl-baselines
        self.reward_space = spaces.Box(
            low=np.array([-1.0, -10.0, -10.0], dtype=np.float32),
            high=np.array([1.0, 0.0, 0.0], dtype=np.float32),
            dtype=np.float32
        )
        self.reward_dim = 3

        # CRITICAL FIX: Add EnvSpec to prevent AttributeError in morl-baselines
        self.spec = EnvSpec(id='Polypharmacy-v0')

    def _row_to_obs(self, row):
        return np.array([
            np.clip(row['age']/100, 0, 1),
            np.clip(row['n_drugs']/30, 0, 1),
            np.clip(row['total_se']/1000, 0, 1),
            np.clip(row['ddi_count']/50, 0, 1)
        ], dtype=np.float32)

    def _row_to_reward(self, row, action):
        # Efficacy: favor 4-6 drugs (common polypharmacy range)
        eff = 1.0 - min(abs(row['n_drugs'] - 5)/25, 1.0)

        # ADR risk: negative penalty for DDIs
        adr = -float(row['ddi_count'])/50.0

        # Tolerability: negative penalty for side effects
        tol = -float(row['total_se'])/1000.0

        # Action effects
        if action == 0:  # Reduce risk
            adr *= 0.8
            tol *= 0.9

        return np.clip([eff, adr, tol],
                      self.reward_space.low,
                      self.reward_space.high)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.idx = np.random.randint(0, self.n)
        row = self.df.iloc[self.idx]
        return self._row_to_obs(row), {}

    def step(self, action):
        row = self.df.iloc[self.idx]
        reward = self._row_to_reward(row, action)
        terminated, truncated = True, False  # One-step episodes
        obs, _ = self.reset()
        return obs, reward, terminated, truncated, {"hadm_id": row['hadm_id']}

# Load your modeling table from Phase 5
model_path = '/content/drive/MyDrive/ML Patent/polypharmacy_project/polypharmacy_model_table.csv'
model = pd.read_csv(model_path)

# Split dataset
hadms = model['hadm_id'].unique()
np.random.shuffle(hadms)
split = int(0.8 * len(hadms))
train_ids, test_ids = hadms[:split], hadms[split:]

train_df = model[model['hadm_id'].isin(train_ids)].reset_index(drop=True)
test_df = model[model['hadm_id'].isin(test_ids)].reset_index(drop=True)

# Create environments
train_env = PolypharmacyEnv(train_df)
test_env = PolypharmacyEnv(test_df)

# Verify the fix
print(f"✅ Environment spec ID: {train_env.spec.id}")
print(f"✅ Reward space: {train_env.reward_space}")
print("✅ Phase 6 Complete: Environment ready for MORL training!")


✅ Environment spec ID: Polypharmacy-v0
✅ Reward space: Box([ -1. -10. -10.], [1. 0. 0.], (3,), float32)
✅ Phase 6 Complete: Environment ready for MORL training!


In [33]:
# import gymnasium as gym
# from gymnasium import spaces
# from gymnasium.envs.registration import EnvSpec  # CRITICAL IMPORT
# import numpy as np
# import pandas as pd

# class PolypharmacyEnv(gym.Env):
#     def __init__(self, df: pd.DataFrame):
#         super().__init__()
#         self.df = df.reset_index(drop=True)
#         self.n = len(self.df)

#         # State: [age, n_drugs, total_se, ddi_count]
#         self.observation_space = spaces.Box(
#             low=0.0, high=1.0, shape=(4,), dtype=np.float32
#         )

#         # Actions: 0=reduce risk, 1=maintain
#         self.action_space = spaces.Discrete(2)

#         # CRITICAL: Define reward_space for morl-baselines
#         self.reward_space = spaces.Box(
#             low=np.array([-1.0, -10.0, -10.0], dtype=np.float32),
#             high=np.array([1.0, 0.0, 0.0], dtype=np.float32),
#             dtype=np.float32
#         )
#         self.reward_dim = 3

#         # CRITICAL FIX: Add EnvSpec to prevent AttributeError in morl-baselines
#         self.spec = EnvSpec(id='Polypharmacy-v0')

#     def _row_to_obs(self, row):
#         return np.array([
#             np.clip(row['age']/100, 0, 1),
#             np.clip(row['n_drugs']/30, 0, 1),
#             np.clip(row['total_se']/1000, 0, 1),
#             np.clip(row['ddi_count']/50, 0, 1)
#         ], dtype=np.float32)

#     def _row_to_reward(self, row, action):
#         # Efficacy: favor 4-6 drugs (common polypharmacy range)
#         eff = 1.0 - min(abs(row['n_drugs'] - 5)/25, 1.0)

#         # ADR risk: negative penalty for DDIs
#         adr = -float(row['ddi_count'])/50.0

#         # Tolerability: negative penalty for side effects
#         tol = -float(row['total_se'])/1000.0

#         # Action effects
#         if action == 0:  # Reduce risk
#             adr *= 0.8
#             tol *= 0.9

#         return np.clip([eff, adr, tol],
#                       self.reward_space.low,
#                       self.reward_space.high)

#     def reset(self, seed=None, options=None):
#         super().reset(seed=seed)
#         self.idx = np.random.randint(0, self.n)
#         row = self.df.iloc[self.idx]
#         return self._row_to_obs(row), {}

#     def step(self, action):
#         row = self.df.iloc[self.idx]
#         reward = self._row_to_reward(row, action)
#         terminated, truncated = True, False  # One-step episodes
#         obs, _ = self.reset()
#         return obs, reward, terminated, truncated, {"hadm_id": row['hadm_id']}

In [12]:
import morl_baselines, pkgutil

for _, mod, _ in pkgutil.walk_packages(
    morl_baselines.__path__, prefix="morl_baselines."
):
    print(mod)


morl_baselines.common
morl_baselines.common.accrued_reward_buffer
morl_baselines.common.buffer
morl_baselines.common.diverse_buffer
morl_baselines.common.evaluation
morl_baselines.common.experiments
morl_baselines.common.model_based
morl_baselines.common.model_based.probabilistic_ensemble
morl_baselines.common.model_based.tabular_model
morl_baselines.common.model_based.utils
morl_baselines.common.morl_algorithm
morl_baselines.common.networks
morl_baselines.common.pareto
morl_baselines.common.performance_indicators
morl_baselines.common.prioritized_buffer
morl_baselines.common.scalarization
morl_baselines.common.utils
morl_baselines.common.weights
morl_baselines.multi_policy
morl_baselines.multi_policy.capql
morl_baselines.multi_policy.capql.capql
morl_baselines.multi_policy.envelope
morl_baselines.multi_policy.envelope.envelope
morl_baselines.multi_policy.gpi_pd
morl_baselines.multi_policy.gpi_pd.gpi_pd
morl_baselines.multi_policy.gpi_pd.gpi_pd_continuous_action
morl_baselines.multi_po

In [13]:
import inspect
import morl_baselines.multi_policy.linear_support.linear_support as ls

print(inspect.getsource(ls))


"""Linear Support implementation."""

import random
from copy import deepcopy
from typing import List, Optional

import cvxpy as cp
import numpy as np
from cvxpy import SolverError
from gymnasium.core import Env

from morl_baselines.common.evaluation import policy_evaluation_mo
from morl_baselines.common.morl_algorithm import MOPolicy
from morl_baselines.common.performance_indicators import hypervolume
from morl_baselines.common.weights import extrema_weights


try:
    import cdd
except ImportError as e:
    raise ImportError(
        "To use this feature, you need to install the optional dependency pycddlib==2.1.6 as stated in pyproject.toml of morl-baselines"
    ) from e


np.set_printoptions(precision=4)


class LinearSupport:
    """Linear Support for computing corner weights when using linear utility functions.

    Implements both

    Optimistic Linear Support (OLS) algorithm:
    Paper: (Section 3.3 of http://roijers.info/pub/thesis.pdf).

    Generalized Policy Improvement L

In [ ]:
# # === Phase 6.5: pycddlib 3.x Compatibility Patch ===
# # Run this AFTER Phases 1-6, BEFORE Phase 7

# import numpy as np
# import cdd
# import morl_baselines.multi_policy.linear_support.linear_support as ls_mod

# # Save original method
# LinearSupport = ls_mod.LinearSupport
# _orig_compute_corner_weights = LinearSupport.compute_corner_weights

# def patched_compute_corner_weights(self):
#     """Patched compute_corner_weights for pycddlib 3.x matrix_from_array API"""
#     # === original pre-processing (unchanged) ===
#     A = np.vstack(self.ccs)
#     A = np.round_(A, 4)
#     A = np.concatenate((A, -np.ones((A.shape[0], 1))), axis=1)

#     A_plus = np.ones((1, A.shape[1])); A_plus[0, -1] = 0
#     A = np.concatenate((A, A_plus), axis=0)
#     A_plus = -np.ones((1, A.shape[1])); A_plus[0, -1] = 0
#     A = np.concatenate((A, A_plus), axis=0)

#     for i in range(self.num_objectives):
#         A_plus = np.zeros((1, A.shape[1])); A_plus[0, i] = -1
#         A = np.concatenate((A, A_plus), axis=0)

#     b = np.zeros(len(self.ccs) + 2 + self.num_objectives)
#     b[len(self.ccs)] = 1
#     b[len(self.ccs) + 1] = -1

#     # === NEW pycddlib 3.x compatible compute_poly_vertices ===
#     def compute_poly_vertices(A, b):
#         b = b.reshape(-1, 1)
#         mat = cdd.matrix_from_array(np.hstack([b, -A]))  # NEW API
#         mat.rep_type = cdd.RepType.INEQUALITY

#         P = cdd.Polyhedron(mat)
#         G = np.array(P.get_generators())

#         verts = []
#         for row in G:
#             if row[0] == 1:  # vertex-type generator
#                 verts.append(row[1:])
#         return verts

#     vertices = compute_poly_vertices(A, b)

#     # === post-processing (unchanged) ===
#     corners = []
#     for v in vertices:
#         cw = np.abs(v[:-1])
#         s = cw.sum()
#         if s > 0:
#             corners.append(cw / s)

#     return corners

# # APPLY PATCH
# LinearSupport.compute_corner_weights = patched_compute_corner_weights

# print("✅ LinearSupport patched successfully for pycddlib 3.x")


In [ ]:
# # === Phase 6.5: SciPy-based Polytope Solver (Python 3.12 Compatible) ===
# # Replaces pycddlib with SciPy.spatial.HalfspaceIntersection
# # Run AFTER Phases 1-6, BEFORE Phase 7

# import numpy as np
# from scipy.spatial import HalfspaceIntersection
# from scipy.spatial.qhull import QhullError
# import morl_baselines.multi_policy.linear_support.linear_support as ls_mod

# # Save original
# _orig_compute_poly_vertices = ls_mod.compute_poly_vertices

# def _scipy_compute_poly_vertices(A, b):
#     """
#     SciPy-based polytope vertex computation.
#     Replaces cddlib's compute_poly_vertices with Qhull.
#     """
#     # Ensure inputs are 2D arrays
#     b = b.reshape(-1, 1)
#     A = np.asarray(A)

#     # Build halfspaces: b + A*x >= 0  ->  A*x + b >= 0
#     # SciPy format: [b, A] where halfspace is b + A*x >= 0
#     halfspaces = np.hstack([b, -A])  # Note: -A because of SciPy's convention

#     # Add feasible interior point (simplex center)
#     # Use the average of constraint points as interior
#     try:
#         # For bounded polytopes, use origin as interior if feasible
#         interior_point = np.zeros(A.shape[1])

#         # If origin fails, use least squares solution
#         try:
#             hs = HalfspaceIntersection(halfspaces, interior_point)
#         except QhullError:
#             # Fallback: solve for interior point via least squares
#             interior_point = np.linalg.lstsq(A, -b.flatten(), rcond=None)[0]
#             hs = HalfspaceIntersection(halfspaces, interior_point)

#         # Extract vertices
#         vertices = hs.intersections

#         # Filter out near-duplicate vertices (numerical tolerance)
#         unique_vertices = []
#         for v in vertices:
#             if not any(np.allclose(v, uv, atol=1e-6) for uv in unique_vertices):
#                 unique_vertices.append(v)

#         return np.array(unique_vertices)

#     except Exception as e:
#         # If all fails, return empty array (no vertices)
#         print(f"Warning: SciPy polytope computation failed: {e}")
#         return np.array([])

# # Apply patch
# ls_mod.compute_poly_vertices = _scipy_compute_poly_vertices

# print("✅ SciPy-based polytope solver patched successfully")
# print("✅ Compatible with Python 3.12, no C extensions, no segfaults")


In [ ]:
# # === CORRECT Phase 6.5: SciPy Patch (Target the Right Function) ===

# import numpy as np
# from scipy.spatial import HalfspaceIntersection, QhullError
# import morl_baselines.multi_policy.linear_support.linear_support as ls_mod

# # Get the LinearSupport class
# LinearSupport = ls_mod.LinearSupport

# # Save the original class method
# _orig_compute_corner_weights = LinearSupport.compute_corner_weights

# def _patched_compute_corner_weights(self):
#     """
#     Patched compute_corner_weights using SciPy instead of pycddlib.
#     This replaces the internal polytope computation.
#     """
#     # === original pre-processing (unchanged) ===
#     A = np.vstack(self.ccs)
#     A = np.round_(A, 4)
#     A = np.concatenate((A, -np.ones((A.shape[0], 1))), axis=1)

#     A_plus = np.ones((1, A.shape[1])); A_plus[0, -1] = 0
#     A = np.concatenate((A, A_plus), axis=0)
#     A_plus = -np.ones((1, A.shape[1])); A_plus[0, -1] = 0
#     A = np.concatenate((A, A_plus), axis=0)

#     for i in range(self.num_objectives):
#         A_plus = np.zeros((1, A.shape[1])); A_plus[0, i] = -1
#         A = np.concatenate((A, A_plus), axis=0)

#     b = np.zeros(len(self.ccs) + 2 + self.num_objectives)
#     b[len(self.ccs)] = 1
#     b[len(self.ccs) + 1] = -1

#     # === NEW: SciPy-based polytope solver (replaces cddlib) ===
#     def compute_poly_vertices_scipy(A, b):
#         """Compute polytope vertices using SciPy.spatial"""
#         b = b.reshape(-1, 1)

#         # Build halfspaces: b + A*x >= 0
#         halfspaces = np.hstack([b, -A])

#         # Add feasible interior point (origin works if feasible)
#         try:
#             interior_point = np.zeros(A.shape[1])
#             hs = HalfspaceIntersection(halfspaces, interior_point)
#         except QhullError:
#             # Fallback: use least squares solution as interior point
#             interior_point = np.linalg.lstsq(A, -b.flatten(), rcond=None)[0]
#             hs = HalfspaceIntersection(halfspaces, interior_point)

#         # Extract unique vertices
#         vertices = hs.intersections
#         unique_vertices = []
#         for v in vertices:
#             if not any(np.allclose(v, uv, atol=1e-6) for uv in unique_vertices):
#                 unique_vertices.append(v)

#         return np.array(unique_vertices)

#     # Compute vertices using SciPy
#     vertices = compute_poly_vertices_scipy(A, b)

#     # === post-processing (unchanged) ===
#     corners = []
#     for v in vertices:
#         cw = np.abs(v[:-1])
#         s = cw.sum()
#         if s > 0:
#             corners.append(cw / s)

#     return corners

# # APPLY PATCH to the CLASS METHOD
# LinearSupport.compute_corner_weights = _patched_compute_corner_weights

# print("✅ LinearSupport.compute_corner_weights patched with SciPy solver")
# print("✅ No pycddlib needed, no segfaults, Python 3.12 compatible")


✅ LinearSupport.compute_corner_weights patched with SciPy solver
✅ No pycddlib needed, no segfaults, Python 3.12 compatible


In [ ]:
# # === CORRECT Phase 6.5: SciPy Patch (Target the Right Function) ===

# import numpy as np
# from scipy.spatial import HalfspaceIntersection, QhullError
# import morl_baselines.multi_policy.linear_support.linear_support as ls_mod

# # Get the LinearSupport class
# LinearSupport = ls_mod.LinearSupport

# # Save the original class method
# _orig_compute_corner_weights = LinearSupport.compute_corner_weights

# def _patched_compute_corner_weights(self):
#     """
#     Patched compute_corner_weights using SciPy instead of pycddlib.
#     This replaces the internal polytope computation.
#     """
#     # === original pre-processing (unchanged) ===
#     A = np.vstack(self.ccs)
#     A = np.round_(A, 4)
#     A = np.concatenate((A, -np.ones((A.shape[0], 1))), axis=1)

#     A_plus = np.ones((1, A.shape[1])); A_plus[0, -1] = 0
#     A = np.concatenate((A, A_plus), axis=0)
#     A_plus = -np.ones((1, A.shape[1])); A_plus[0, -1] = 0
#     A = np.concatenate((A, A_plus), axis=0)

#     for i in range(self.num_objectives):
#         A_plus = np.zeros((1, A.shape[1])); A_plus[0, i] = -1
#         A = np.concatenate((A, A_plus), axis=0)

#     b = np.zeros(len(self.ccs) + 2 + self.num_objectives)
#     b[len(self.ccs)] = 1
#     b[len(self.ccs) + 1] = -1

#     # === NEW: SciPy-based polytope solver (replaces cddlib) ===
#     def compute_poly_vertices_scipy(A, b):
#         """Compute polytope vertices using SciPy.spatial"""
#         b = b.reshape(-1, 1)

#         # Build halfspaces: b + A*x >= 0
#         halfspaces = np.hstack([b, -A])

#         # Add feasible interior point (origin works if feasible)
#         try:
#             interior_point = np.zeros(A.shape[1])
#             hs = HalfspaceIntersection(halfspaces, interior_point)
#         except QhullError:
#             # Fallback: use least squares solution as interior point
#             interior_point = np.linalg.lstsq(A, -b.flatten(), rcond=None)[0]
#             hs = HalfspaceIntersection(halfspaces, interior_point)

#         # Extract unique vertices
#         vertices = hs.intersections
#         unique_vertices = []
#         for v in vertices:
#             if not any(np.allclose(v, uv, atol=1e-6) for uv in unique_vertices):
#                 unique_vertices.append(v)

#         return np.array(unique_vertices)

#     # Compute vertices using SciPy
#     vertices = compute_poly_vertices_scipy(A, b)

#     # === post-processing (unchanged) ===
#     corners = []
#     for v in vertices:
#         cw = np.abs(v[:-1])
#         s = cw.sum()
#         if s > 0:
#             corners.append(cw / s)

#     return corners

# # APPLY PATCH to the CLASS METHOD
# LinearSupport.compute_corner_weights = _patched_compute_corner_weights

# print("✅ LinearSupport.compute_corner_weights patched with SciPy solver")
# print("✅ No pycddlib needed, no segfaults, Python 3.12 compatible")


✅ LinearSupport.compute_corner_weights patched with SciPy solver
✅ No pycddlib needed, no segfaults, Python 3.12 compatible


In [ ]:
# import morl_baselines.multi_policy.linear_support.linear_support as ls_mod

# print("Module attributes:")
# print([attr for attr in dir(ls_mod) if not attr.startswith('_')])

# print("\nLinearSupport class methods:")
# print([attr for attr in dir(ls_mod.LinearSupport) if not attr.startswith('_')])

# print("\nLinearSupport.compute_corner_weights exists?",
#       hasattr(ls_mod.LinearSupport, 'compute_corner_weights'))

# print("\nModule-level compute_poly_vertices exists?",
#       hasattr(ls_mod, 'compute_poly_vertices'))


Module attributes:
['Env', 'LinearSupport', 'List', 'MOPolicy', 'Optional', 'SolverError', 'cdd', 'cp', 'deepcopy', 'extrema_weights', 'hypervolume', 'np', 'policy_evaluation_mo', 'random']

LinearSupport class methods:
['add_solution', 'compute_corner_weights', 'ended', 'get_corner_weights', 'get_weight_support', 'gpi_ls_priority', 'is_dominated', 'max_scalarized_value', 'max_value_lp', 'next_weight', 'ols_priority', 'remove_obsolete_values', 'remove_obsolete_weights']

LinearSupport.compute_corner_weights exists? True

Module-level compute_poly_vertices exists? False


In [14]:
# === Phase 6.5: Robust SciPy-based Polytope Solver (Handles Numerical Issues) ===

import numpy as np
from scipy.spatial import HalfspaceIntersection
import morl_baselines.multi_policy.linear_support.linear_support as ls_mod

# Get the LinearSupport class
LinearSupport = ls_mod.LinearSupport

# Save original method
_orig_compute_corner_weights = LinearSupport.compute_corner_weights

def _patched_compute_corner_weights(self):
    """
    Patched compute_corner_weights with robust SciPy polytope solver.
    Handles QhullError with epsilon padding and fallback strategies.
    """
    # === original pre-processing (unchanged) ===
    A = np.vstack(self.ccs)
    A = np.round_(A, 4)
    A = np.concatenate((A, -np.ones((A.shape[0], 1))), axis=1)

    A_plus = np.ones((1, A.shape[1])); A_plus[0, -1] = 0
    A = np.concatenate((A, A_plus), axis=0)
    A_plus = -np.ones((1, A.shape[1])); A_plus[0, -1] = 0
    A = np.concatenate((A, A_plus), axis=0)

    for i in range(self.num_objectives):
        A_plus = np.zeros((1, A.shape[1])); A_plus[0, i] = -1
        A = np.concatenate((A, A_plus), axis=0)

    b = np.zeros(len(self.ccs) + 2 + self.num_objectives)
    b[len(self.ccs)] = 1
    b[len(self.ccs) + 1] = -1

    # === ROBUST: SciPy-based polytope solver with epsilon padding ===
    def compute_poly_vertices_robust(A, b, epsilon=1e-6, max_iterations=10):
        """
        Compute polytope vertices with robust interior point finding.
        Adds epsilon padding to ensure interior point is strictly feasible.
        """
        b = b.reshape(-1, 1)
        halfspaces = np.hstack([b, -A])
        n_halfspaces, n_dim = halfspaces.shape

        # Strategy 1: Try origin with epsilon shift
        for attempt in range(max_iterations):
            try:
                # Add small epsilon to move away from boundaries
                offset = epsilon * (attempt + 1)
                interior_point = np.full(n_dim - 1, offset)  # -1 because of [b, A] format

                # Ensure it's strictly feasible: check all halfspaces
                # halfspace: b + A*x >= 0
                for i in range(n_halfspaces):
                    val = halfspaces[i, 0] + np.dot(halfspaces[i, 1:], interior_point)
                    if val <= 0:
                        raise ValueError(f"Halfspace {i} violated: {val}")

                hs = HalfspaceIntersection(halfspaces, interior_point)
                break  # Success!

            except Exception as e:
                if attempt == max_iterations - 1:
                    # Strategy 2: Use linear programming relaxation
                    print(f"Warning: Standard interior point failed ({e}), using LP relaxation")

                    # Solve min ||x||² subject to A*x + b >= epsilon
                    # This is a simple quadratic programming approximation
                    from scipy.optimize import minimize

                    def objective(x):
                        return np.sum(x**2)

                    def constraint(i):
                        return lambda x: halfspaces[i, 0] + np.dot(halfspaces[i, 1:], x) - epsilon

                    constraints = [{'type': 'ineq', 'fun': constraint(i)}
                                  for i in range(n_halfspaces)]

                    # Start from origin
                    result = minimize(objective, np.zeros(n_dim - 1),
                                    constraints=constraints, method='SLSQP')

                    if result.success:
                        interior_point = result.x
                        hs = HalfspaceIntersection(halfspaces, interior_point)
                    else:
                        # Strategy 3: Return empty (degenerate polytope)
                        print("Warning: Polytope is degenerate or infeasible, returning empty")
                        return np.array([])
                else:
                    # Try again with larger epsilon
                    continue

        # Extract vertices and remove duplicates
        vertices = hs.intersections
        unique_vertices = []

        for v in vertices:
            # Check if vertex is close to any existing unique vertex
            is_duplicate = False
            for uv in unique_vertices:
                if np.linalg.norm(v - uv) < epsilon * 10:
                    is_duplicate = True
                    break
            if not is_duplicate:
                unique_vertices.append(v)

        return np.array(unique_vertices)

    # Compute vertices
    vertices = compute_poly_vertices_robust(A, b)

    # === post-processing (unchanged) ===
    corners = []
    for v in vertices:
        cw = np.abs(v[:-1])
        s = cw.sum()
        if s > 0:
            corners.append(cw / s)

    return corners

# APPLY PATCH to CLASS METHOD
LinearSupport.compute_corner_weights = _patched_compute_corner_weights

print("✅ Robust SciPy-based LinearSupport patch applied")
print("✅ Handles numerical precision issues with epsilon padding")


✅ Robust SciPy-based LinearSupport patch applied
✅ Handles numerical precision issues with epsilon padding


Phase 7: MORL Training and Evaluation
Goal: Train a multi-objective agent and visualize the Pareto front.

In [ ]:
# # === Phase 7: MORL Training, Evaluation & Saving ===
# from morl_baselines.multi_policy.gpi_pd.gpi_pd import GPIPD
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import json
# import os
# from datetime import datetime

# # Ensure save directory exists
# save_dir = '/content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results'
# os.makedirs(save_dir, exist_ok=True)
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# # Initialize agent
# agent = GPIPD(
#     env=train_env,
#     learning_rate=3e-4,
#     gamma=0.99,
#     batch_size=256,
#     net_arch=[128, 128],
#     buffer_size=int(5e4),
#     initial_epsilon=1.0,
#     final_epsilon=0.05,
#     epsilon_decay_steps=20000,
#     learning_starts=1000,
#     gradient_updates=1,
#     target_net_update_freq=500,
#     tau=1.0,
#     log=False,
#     dyna=False
# )
# ref_point = train_env.reward_space.low.copy()

# # Train agent
# print("Starting training...")
# agent.train(
#     total_timesteps=30000,
#     ref_point=ref_point,
#     eval_env=test_env,
#     eval_freq=5000
# )
# print("✅ Training complete!")

# # Save trained model
# model_path = os.path.join(save_dir, f'gpipd_model_{timestamp}.pkl')
# agent.save(model_path)
# print(f"💾 Saved trained model: {model_path}")

# # Evaluation function
# def evaluate(agent, env, n_episodes=200):
#     out = []
#     episode_info = []
#     for i in range(n_episodes):
#         obs, _ = env.reset()
#         done = False
#         total = np.zeros(3, dtype=np.float32)
#         while not done:
#             action, _ = agent.predict(obs, deterministic=True)
#             obs, r, done, trunc, info = env.step(action)
#             total += r
#         out.append(total)
#         episode_info.append(info)
#     return np.vstack(out), episode_info

# # Run evaluation
# print("Evaluating agent...")
# results, episode_infos = evaluate(agent, test_env)

# # Convert to DataFrame for analysis
# res_df = pd.DataFrame(results, columns=['efficacy','neg_ddi','neg_tol'])
# res_df['hadm_id'] = [info.get('hadm_id') for info in episode_infos]

# # Save evaluation results
# results_path = os.path.join(save_dir, f'evaluation_results_{timestamp}.csv')
# res_df.to_csv(results_path, index=False)
# print(f"💾 Saved evaluation results: {results_path}")

# # Save hyperparameters
# hyperparams = {
#     'learning_rate': 3e-4,
#     'gamma': 0.99,
#     'batch_size': 256,
#     'net_arch': [128, 128],
#     'buffer_size': int(5e4),
#     'epsilon_decay_steps': 20000,
#     'total_timesteps': 30000,
#     'n_eval_episodes': 200,
#     'ref_point': ref_point.tolist(),
#     'dyna': False,
#     'pycddlib_patch': 'matrix_from_array API'
# }
# hyperparams_path = os.path.join(save_dir, f'hyperparameters_{timestamp}.json')
# with open(hyperparams_path, 'w') as f:
#     json.dump(hyperparams, f, indent=2)
# print(f"💾 Saved hyperparameters: {hyperparams_path}")

# # Generate and save Pareto front plot
# plt.figure(figsize=(10, 8))
# scatter = plt.scatter(res_df['efficacy'], res_df['neg_ddi'],
#                      c=res_df['neg_tol'], cmap='viridis', alpha=0.6, s=50)
# plt.xlabel('Efficacy Proxy (Higher Better)', fontsize=12)
# plt.ylabel('DDI Risk (Less Negative Better)', fontsize=12)
# plt.colorbar(scatter, label='Tolerability (Less Negative Better)')
# plt.title('Pareto-optimal Prescription Policies\nMulti-Objective RL on MIMIC-III Polypharmacy',
#           fontsize=14, fontweight='bold')
# plt.grid(True, alpha=0.3)

# # Add statistics text
# mean_vals = res_df.mean()
# std_vals = res_df.std()
# stats_text = f"Mean Efficacy: {mean_vals['efficacy']:.3f} ± {std_vals['efficacy']:.3f}\nMean DDI: {mean_vals['neg_ddi']:.3f} ± {std_vals['neg_ddi']:.3f}\nMean Tolerability: {mean_vals['neg_tol']:.3f} ± {std_vals['neg_tol']:.3f}"
# plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes,
#          fontsize=10, verticalalignment='top',
#          bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# plt.tight_layout()

# # Save plot
# plot_path = os.path.join(save_dir, f'pareto_front_{timestamp}.png')
# plt.savefig(plot_path, dpi=300, bbox_inches='tight')
# print(f"💾 Saved Pareto plot: {plot_path}")
# plt.show()

# # Summary statistics
# print("\n=== Final Performance Summary ===")
# print(f"Mean ± Std (Efficacy): {mean_vals['efficacy']:.4f} ± {std_vals['efficacy']:.4f}")
# print(f"Mean ± Std (DDI Risk): {mean_vals['neg_ddi']:.4f} ± {std_vals['neg_ddi']:.4f}")
# print(f"Mean ± Std (Tolerability): {mean_vals['neg_tol']:.4f} ± {std_vals['neg_tol']:.4f}")

# # Save summary report
# summary = {
#     'timestamp': timestamp,
#     'n_eval_episodes': len(res_df),
#     'mean_efficacy': float(mean_vals['efficacy']),
#     'std_efficacy': float(std_vals['efficacy']),
#     'mean_ddi': float(mean_vals['neg_ddi']),
#     'std_ddi': float(std_vals['neg_ddi']),
#     'mean_tolerability': float(mean_vals['neg_tol']),
#     'std_tolerability': float(std_vals['neg_tol']),
#     'model_path': model_path,
#     'results_path': results_path,
#     'plot_path': plot_path,
#     'hyperparams_path': hyperparams_path,
#     'model_based_learning': False,
#     'pycddlib_patch_applied': True
# }
# summary_path = os.path.join(save_dir, f'summary_{timestamp}.json')
# with open(summary_path, 'w') as f:
#     json.dump(summary, f, indent=2)
# print(f"💾 Saved summary report: {summary_path}")

# print(f"\n🎉 Phase 7 Complete! All artifacts saved to: {save_dir}")


Starting training...
CCS: [] CCS size: 0
Next weight: [1. 0. 0.]
Next weight vector: [1. 0. 0.]
Adding value: [ 0.16  -0.144 -1.937] to CCS.
Adding value: [ 0.096   0.     -2.7441] to CCS.
Adding value: [ 0.128   0.     -2.0666] to CCS.
removed value [ 0.096   0.     -2.7441]
W_corner: [] W_corner size: 0
CCS: [array([ 0.16 , -0.144, -1.937], dtype=float32), array([ 0.128 ,  0.    , -2.0666], dtype=float32)] CCS size: 2
There are no corner weights in the queue. Returning None.
✅ Training complete!


AttributeError: 'GPIPD' object has no attribute 'experiment_name'

In [ ]:
# # === Phase 7: MORL Training, Evaluation & Saving (COMPLETE, FIXED) ===
# from morl_baselines.multi_policy.gpi_pd.gpi_pd import GPIPD
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import json
# import os
# from datetime import datetime

# # Ensure save directory exists
# save_dir = '/content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results'
# os.makedirs(save_dir, exist_ok=True)
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# # Initialize agent
# agent = GPIPD(
#     env=train_env,
#     learning_rate=3e-4,
#     gamma=0.99,
#     batch_size=256,
#     net_arch=[128, 128],
#     buffer_size=int(5e4),
#     initial_epsilon=1.0,
#     final_epsilon=0.05,
#     epsilon_decay_steps=20000,
#     learning_starts=1000,
#     gradient_updates=1,
#     target_net_update_freq=500,
#     tau=1.0,
#     log=False,
#     dyna=False
# )
# ref_point = train_env.reward_space.low.copy()

# # Train agent
# print("Starting training...")
# agent.train(
#     total_timesteps=30000,
#     ref_point=ref_point,
#     eval_env=test_env,
#     eval_freq=5000
# )
# print("✅ Training complete!")

# # FIX: Add missing attributes that morl-baselines expects
# agent.experiment_name = "gpipd_polypharmacy"
# agent.save_dir = save_dir

# # Save trained model
# model_path = os.path.join(save_dir, f'gpipd_model_{timestamp}')
# agent.save(model_path)
# print(f"💾 Saved trained model: {model_path}.tar")

# # Evaluation function
# def evaluate(agent, env, n_episodes=200):
#     out = []
#     episode_info = []
#     for i in range(n_episodes):
#         obs, _ = env.reset()
#         done = False
#         total = np.zeros(3, dtype=np.float32)
#         while not done:
#             action, _ = agent.predict(obs, deterministic=True)
#             obs, r, done, trunc, info = env.step(action)
#             total += r
#         out.append(total)
#         episode_info.append(info)
#     return np.vstack(out), episode_info

# # Run evaluation
# print("Evaluating agent...")
# results, episode_infos = evaluate(agent, test_env)

# # Convert to DataFrame for analysis
# res_df = pd.DataFrame(results, columns=['efficacy','neg_ddi','neg_tol'])
# res_df['hadm_id'] = [info.get('hadm_id') for info in episode_infos]

# # Save evaluation results
# results_path = os.path.join(save_dir, f'evaluation_results_{timestamp}.csv')
# res_df.to_csv(results_path, index=False)
# print(f"💾 Saved evaluation results: {results_path}")

# # Save hyperparameters
# hyperparams = {
#     'learning_rate': 3e-4,
#     'gamma': 0.99,
#     'batch_size': 256,
#     'net_arch': [128, 128],
#     'buffer_size': int(5e4),
#     'epsilon_decay_steps': 20000,
#     'total_timesteps': 30000,
#     'n_eval_episodes': 200,
#     'ref_point': ref_point.tolist(),
#     'dyna': False,
#     'pycddlib_patch': 'scipy_based'
# }
# hyperparams_path = os.path.join(save_dir, f'hyperparameters_{timestamp}.json')
# with open(hyperparams_path, 'w') as f:
#     json.dump(hyperparams, f, indent=2)
# print(f"💾 Saved hyperparameters: {hyperparams_path}")

# # Generate and save Pareto front plot
# plt.figure(figsize=(10, 8))
# scatter = plt.scatter(res_df['efficacy'], res_df['neg_ddi'],
#                      c=res_df['neg_tol'], cmap='viridis', alpha=0.6, s=50)
# plt.xlabel('Efficacy Proxy (Higher Better)', fontsize=12)
# plt.ylabel('DDI Risk (Less Negative Better)', fontsize=12)
# plt.colorbar(scatter, label='Tolerability (Less Negative Better)')
# plt.title('Pareto-optimal Prescription Policies\nMulti-Objective RL on MIMIC-III Polypharmacy',
#           fontsize=14, fontweight='bold')
# plt.grid(True, alpha=0.3)

# # Add statistics text
# mean_vals = res_df.mean()
# std_vals = res_df.std()
# stats_text = f"Mean Efficacy: {mean_vals['efficacy']:.3f} ± {std_vals['efficacy']:.3f}\nMean DDI: {mean_vals['neg_ddi']:.3f} ± {std_vals['neg_ddi']:.3f}\nMean Tolerability: {mean_vals['neg_tol']:.3f} ± {std_vals['neg_tol']:.3f}"
# plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes,
#          fontsize=10, verticalalignment='top',
#          bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# plt.tight_layout()

# # Save plot
# plot_path = os.path.join(save_dir, f'pareto_front_{timestamp}.png')
# plt.savefig(plot_path, dpi=300, bbox_inches='tight')
# print(f"💾 Saved Pareto plot: {plot_path}")
# plt.show()

# # Summary statistics
# print("\n=== Final Performance Summary ===")
# print(f"Mean ± Std (Efficacy): {mean_vals['efficacy']:.4f} ± {std_vals['efficacy']:.4f}")
# print(f"Mean ± Std (DDI Risk): {mean_vals['neg_ddi']:.4f} ± {std_vals['neg_ddi']:.4f}")
# print(f"Mean ± Std (Tolerability): {mean_vals['neg_tol']:.4f} ± {std_vals['neg_tol']:.4f}")

# # Save summary report
# summary = {
#     'timestamp': timestamp,
#     'n_eval_episodes': len(res_df),
#     'mean_efficacy': float(mean_vals['efficacy']),
#     'std_efficacy': float(std_vals['efficacy']),
#     'mean_ddi': float(mean_vals['neg_ddi']),
#     'std_ddi': float(std_vals['neg_ddi']),
#     'mean_tolerability': float(mean_vals['neg_tol']),
#     'std_tolerability': float(std_vals['neg_tol']),
#     'model_path': model_path,
#     'results_path': results_path,
#     'plot_path': plot_path,
#     'hyperparams_path': hyperparams_path,
#     'model_based_learning': False,
#     'pycddlib_patch_applied': True
# }
# summary_path = os.path.join(save_dir, f'summary_{timestamp}.json')
# with open(summary_path, 'w') as f:
#     json.dump(summary, f, indent=2)
# print(f"💾 Saved summary report: {summary_path}")

# print(f"\n🎉 Phase 7 Complete! All artifacts saved to: {save_dir}")


Starting training...
CCS: [] CCS size: 0
Next weight: [1. 0. 0.]
Next weight vector: [1. 0. 0.]
Adding value: [ 0.12   -0.064  -1.6117] to CCS.
Adding value: [ 0.096  -0.144  -1.7006] to CCS.
Value [ 0.096  -0.144  -1.7006] is dominated. Discarding.
Adding value: [ 0.464   0.     -0.8406] to CCS.
removed value [ 0.12   -0.064  -1.6117]
W_corner: [] W_corner size: 0
CCS: [array([ 0.464 ,  0.    , -0.8406], dtype=float32)] CCS size: 1
There are no corner weights in the queue. Returning None.
✅ Training complete!
💾 Saved trained model: /content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/gpipd_model_20251117_172144.tar
Evaluating agent...


AttributeError: 'GPIPD' object has no attribute 'predict'

In [35]:
# === Phase 7a: Training (Conditional) ===
from morl_baselines.multi_policy.gpi_pd.gpi_pd import GPIPD
import numpy as np
import os
from datetime import datetime

# Configuration
save_dir = '/content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results'
os.makedirs(save_dir, exist_ok=True)

# Check if model already exists
existing_models = [f for f in os.listdir(save_dir) if f.startswith('gpipd_model_') and f.endswith('.tar')]
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

if existing_models:
    # Use latest model
    latest_model = sorted(existing_models)[-1]
    model_timestamp = latest_model.replace('gpipd_model_', '').replace('.tar', '')
    print(f"✅ Found existing model: {latest_model}")
    print("⏭️  Skipping training...")

    # We'll just load this model in Phase 7b
    agent = None  # Placeholder

else:
    # No model found - train from scratch
    print("🔄 No model found. Starting training...")

    # Initialize agent
    agent = GPIPD(
        env=train_env,
        learning_rate=3e-4,
        gamma=0.99,
        batch_size=256,
        net_arch=[128, 128],
        buffer_size=int(5e4),
        initial_epsilon=1.0,
        final_epsilon=0.05,
        epsilon_decay_steps=20000,
        learning_starts=1000,
        gradient_updates=1,
        target_net_update_freq=500,
        tau=1.0,
        log=False,
        dyna=False
    )
    ref_point = train_env.reward_space.low.copy()

    # Train
    print("Starting training...")
    agent.train(
        total_timesteps=30000,
        ref_point=ref_point,
        eval_env=test_env,
        eval_freq=5000
    )
    print("✅ Training complete!")

    # FIX: Add missing attributes
    agent.experiment_name = "gpipd_polypharmacy"
    agent.save_dir = save_dir

    # Save model
    model_path = os.path.join(save_dir, f'gpipd_model_{timestamp}')
    agent.save(model_path)
    print(f"💾 Saved trained model: {model_path}.tar")

    # Store for Phase 7b
    agent = agent  # Keep reference

print("\n✅ Phase 7a complete. Ready for evaluation (Phase 7b).")


✅ Found existing model: gpipd_model_20251117_175812.tar
⏭️  Skipping training...

✅ Phase 7a complete. Ready for evaluation (Phase 7b).


In [36]:
!ls -lha /content/drive/MyDrive/ML\ Patent/polypharmacy_project/phase7_results


total 4.2M
-rw------- 1 root root 1.9K Nov 18 05:26 clinical_interp.md
-rw------- 1 root root 5.0K Nov 18 04:58 evaluation_results_20251117_175812.csv
-rw------- 1 root root 4.0M Nov 17 17:58 gpipd_model_20251117_175812.tar
-rw------- 1 root root  269 Nov 18 04:58 hyperparameters_20251117_175812.json
-rw------- 1 root root 1.9K Nov 18 05:26 manuscript.md
-rw------- 1 root root 179K Nov 18 04:58 pareto_front_20251117_175812.png
drwx------ 2 root root 4.0K Nov 18 04:49 phase8_results_20251118_044945
drwx------ 2 root root 4.0K Nov 18 04:57 phase8_results_20251118_045718
drwx------ 2 root root 4.0K Nov 18 05:24 phase8_results_20251118_050009
-rw------- 1 root root  884 Nov 18 04:58 summary_20251117_175812.json


In [39]:
# === Phase 7a: Training or Loading + Clean Save ===
from morl_baselines.multi_policy.gpi_pd.gpi_pd import GPIPD
import numpy as np
import torch
import os
from datetime import datetime

save_dir = "/content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results"
os.makedirs(save_dir, exist_ok=True)

# Detect existing models
existing_models = [f for f in os.listdir(save_dir) if f.startswith("gpipd_model_") and f.endswith(".tar")]
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

trained_now = False

# === CASE 1: MODEL EXISTS ===
if existing_models:
    latest_tar = sorted(existing_models)[-1]
    print(f"📦 Found existing model: {latest_tar}")

    # Full file path including .tar
    model_path = os.path.join(save_dir, latest_tar)

    print(f"➡️  Loading GPIPD model from: {model_path}")

    # Recreate GPIPD agent architecture (required before load)
    agent = GPIPD(
        env=train_env,
        learning_rate=3e-4,
        gamma=0.99,
        batch_size=256,
        net_arch=[128, 128],
        buffer_size=int(5e4),
        initial_epsilon=0.05,
        final_epsilon=0.05,
        log=False,
        dyna=False
    )

    # IMPORTANT: load the .tar directly
    agent.load(model_path)

# === CASE 2: NO MODEL → TRAIN NEW ===
else:
    print("🚀 No model found. Training new GPIPD...")

    agent = GPIPD(
        env=train_env,
        learning_rate=3e-4,
        gamma=0.99,
        batch_size=256,
        net_arch=[128, 128],
        buffer_size=int(5e4),
        initial_epsilon=1.0,
        final_epsilon=0.05,
        epsilon_decay_steps=20000,
        learning_starts=1000,
        gradient_updates=1,
        target_net_update_freq=500,
        tau=1.0,
        log=False,
        dyna=False
    )

    ref_point = train_env.reward_space.low.copy()
    agent.train(
        total_timesteps=30000,
        ref_point=ref_point,
        eval_env=test_env,
        eval_freq=5000
    )

    agent.experiment_name = "gpipd_polypharmacy"
    agent.save_dir = save_dir

    # Save with .tar
    model_path = os.path.join(save_dir, f"gpipd_model_{timestamp}.tar")
    agent.save(model_path.replace(".tar", ""))   # GPIPD.save adds .tar automatically

    print(f"💾 Saved full model: {model_path}")
    trained_now = True

# =====================================================
# === CLEAN MODEL EXPORT FOR STREAMLIT ===============
# =====================================================

print("🔧 Extracting q_networks + weight_support…")

clean_path = "/content/gpipd_model_clean.pt"

torch.save({
    "q_networks": [q.state_dict() for q in agent.q_networks],
    "weight_support": agent.weight_support,
}, clean_path)

print(f"✅ Saved clean deployable model: {clean_path}")
print("Phase 7a complete!")


📦 Found existing model: gpipd_model_20251117_175812.tar
➡️  Loading GPIPD model from: /content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/gpipd_model_20251117_175812.tar
🔧 Extracting q_networks + weight_support…


AttributeError: 'GPIPD' object has no attribute 'q_networks'

In [40]:
print("🔧 Extracting q_nets + weight_support…")

clean_path = "/content/gpipd_model_clean.pt"

torch.save({
    "q_nets": [q.state_dict() for q in agent.q_nets],   # FIXED
    "weight_support": agent.weight_support,             # OK
}, clean_path)

print(f"✅ Saved clean deployable model: {clean_path}")


🔧 Extracting q_nets + weight_support…
✅ Saved clean deployable model: /content/gpipd_model_clean.pt


In [ ]:
# import torch

# # Force-save model (GPIPD uses policy_net + target_net + replay buffer)
# forced_model_path = model_path + ".pth"

# torch.save({
#     "policy_net": agent.policy_net.state_dict(),
#     "target_net": agent.target_net.state_dict(),
#     "replay_buffer": agent.replay_buffer,   # stored as Python object
#     "epsilon": agent.epsilon,
#     "iteration": agent.iteration,
# }, forced_model_path)

# print(f"🔥 FORCE-SAVED model to: {forced_model_path}")


AttributeError: 'GPIPD' object has no attribute 'policy_net'

In [ ]:
[type(v) for v in agent.__dict__.values()]


[__main__.PolypharmacyEnv,
 tuple,
 int,
 gymnasium.spaces.discrete.Discrete,
 tuple,
 numpy.int64,
 int,
 torch.device,
 int,
 int,
 NoneType,
 numpy.random._generator.Generator,
 NoneType,
 float,
 float,
 numpy.float64,
 int,
 float,
 float,
 int,
 float,
 NoneType,
 bool,
 int,
 list,
 int,
 int,
 int,
 int,
 float,
 bool,
 list,
 list,
 torch.optim.adam.Adam,
 bool,
 bool,
 morl_baselines.common.prioritized_buffer.PrioritizedReplayBuffer,
 float,
 float,
 bool,
 list,
 NoneType,
 NoneType,
 function,
 int,
 bool,
 int,
 int,
 int,
 int,
 int,
 int,
 float,
 float,
 bool,
 list,
 list,
 str,
 str]

In [ ]:
print(dir(agent))


['_MOPolicy__report', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_act', '_envelope_target', '_reset_priorities', '_rollout_dynamics', '_sample_batch_experiences', 'action_dim', 'action_shape', 'action_space', 'alpha', 'batch_size', 'buffer_size', 'close_wandb', 'device', 'drop_rate', 'dyna', 'dynamics', 'dynamics_buffer', 'dynamics_buffer_size', 'dynamics_ensemble_size', 'dynamics_net_arch', 'dynamics_normalize_inputs', 'dynamics_num_elites', 'dynamics_rollout_batch_size', 'dynamics_rollout_freq', 'dynamics_rollout_len', 'dynamics_rollout_starts', 'dynamics_train_freq', 'dynamics_uncertainty_threshold', 'env', 'epsilon', 'eps

In [ ]:
import inspect
print(inspect.getsource(agent.__class__))


class GPIPD(MOPolicy, MOAgent):
    """GPI-PD Algorithm.

    Sample-Efficient Multi-Objective Learning via Generalized Policy Improvement Prioritization
    Lucas N. Alegre, Ana L. C. Bazzan, Diederik M. Roijers, Ann Nowé, Bruno C. da Silva
    AAMAS 2023
    Paper: https://arxiv.org/abs/2301.07784
    """

    def __init__(
        self,
        env,
        learning_rate: float = 3e-4,
        initial_epsilon: float = 0.01,
        final_epsilon: float = 0.01,
        epsilon_decay_steps: int = None,  # None == fixed epsilon
        tau: float = 1.0,
        target_net_update_freq: int = 1000,  # ignored if tau != 1.0
        buffer_size: int = int(1e6),
        net_arch: List = [256, 256, 256, 256],
        num_nets: int = 2,
        batch_size: int = 128,
        learning_starts: int = 100,
        gradient_updates: int = 20,
        gamma: float = 0.99,
        max_grad_norm: Optional[float] = None,
        use_gpi: bool = True,
        dyna: bool = True,
        per: bool = True

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

agent.experiment_name = f"gpipd_model_{timestamp}"
agent.save_dir = save_dir

# Save the model
agent.save(save_dir=save_dir, save_replay_buffer=True)

# Final absolute path
model_file = f"{save_dir}/{agent.experiment_name}.tar"
print("🔥 Saved model to:", model_file)

# Verify
!ls -lha "$save_dir"


🔥 Saved model to: /content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/gpipd_model_20251117_175812.tar
total 4.0M
-rw------- 1 root root 4.0M Nov 17 17:58 gpipd_model_20251117_175812.tar


In [4]:
# === Phase 7b: Evaluation (Loads Model If Not Already Loaded) ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from datetime import datetime
# Load the latest model
latest_model = sorted(existing_models)[-1]
model_path = os.path.join(save_dir, latest_model)   # FIXED

timestamp = latest_model.replace('gpipd_model_', '').replace('.tar', '')

print(f"Loading model: {latest_model}...")

agent = GPIPD(
    env=train_env,
    learning_rate=3e-4,
    gamma=0.99,
    batch_size=256,
    net_arch=[128, 128],
    buffer_size=int(5e4),
    initial_epsilon=0.05,
    final_epsilon=0.05,
    log=False,
    dyna=False
)

# Load trained weights
agent.load(path=model_path)
print(f"✅ Model loaded from: {model_path}")

# === FIX: Proper GPIPD predict method (no policy_net required) ===

w_eval = np.ones(agent.reward_dim, dtype=np.float32) / agent.reward_dim

def patched_predict(agent, obs, deterministic=True):
    return agent.eval(obs, w_eval), {}

agent.predict = lambda obs, deterministic=True: patched_predict(agent, obs, deterministic)

# Evaluation function
def evaluate(agent, env, n_episodes=200):
    out = []
    episode_info = []
    for i in range(n_episodes):
        obs, _ = env.reset()
        done = False
        total = np.zeros(3, dtype=np.float32)
        while not done:
            action, _ = agent.predict(obs, deterministic=True)
            obs, r, done, trunc, info = env.step(action)
            total += r
        out.append(total)
        episode_info.append(info)
    return np.vstack(out), episode_info

# Run evaluation
print("Evaluating agent...")
results, episode_infos = evaluate(agent, test_env)

# Convert to DataFrame for analysis
res_df = pd.DataFrame(results, columns=['efficacy','neg_ddi','neg_tol'])
res_df['hadm_id'] = [info.get('hadm_id') for info in episode_infos]

# Save evaluation results
results_path = os.path.join(save_dir, f'evaluation_results_{timestamp}.csv')
res_df.to_csv(results_path, index=False)
print(f"💾 Saved evaluation results: {results_path}")

# Save hyperparameters
hyperparams = {
    'learning_rate': 3e-4,
    'gamma': 0.99,
    'batch_size': 256,
    'net_arch': [128, 128],
    'buffer_size': int(5e4),
    'epsilon_decay_steps': 20000,
    'total_timesteps': 30000,
    'n_eval_episodes': 200,
    'ref_point': train_env.reward_space.low.tolist(),
    'dyna': False,
    'pycddlib_patch': 'scipy_based'
}
hyperparams_path = os.path.join(save_dir, f'hyperparameters_{timestamp}.json')
with open(hyperparams_path, 'w') as f:
    json.dump(hyperparams, f, indent=2)
print(f"💾 Saved hyperparameters: {hyperparams_path}")

# Generate and save Pareto front plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(res_df['efficacy'], res_df['neg_ddi'],
                     c=res_df['neg_tol'], cmap='viridis', alpha=0.6, s=50)
plt.xlabel('Efficacy Proxy (Higher Better)', fontsize=12)
plt.ylabel('DDI Risk (Less Negative Better)', fontsize=12)
plt.colorbar(scatter, label='Tolerability (Less Negative Better)')
plt.title('Pareto-optimal Prescription Policies\nMulti-Objective RL on MIMIC-III Polypharmacy',
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Add statistics text
mean_vals = res_df.mean()
std_vals = res_df.std()
stats_text = f"Mean Efficacy: {mean_vals['efficacy']:.3f} ± {std_vals['efficacy']:.3f}\nMean DDI: {mean_vals['neg_ddi']:.3f} ± {std_vals['neg_ddi']:.3f}\nMean Tolerability: {mean_vals['neg_tol']:.3f} ± {std_vals['neg_tol']:.3f}"
plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes,
         fontsize=10, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()

# Save plot
plot_path = os.path.join(save_dir, f'pareto_front_{timestamp}.png')
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"💾 Saved Pareto plot: {plot_path}")
plt.show()

# Summary statistics
print("\n=== Final Performance Summary ===")
print(f"Mean ± Std (Efficacy): {mean_vals['efficacy']:.4f} ± {std_vals['efficacy']:.4f}")
print(f"Mean ± Std (DDI Risk): {mean_vals['neg_ddi']:.4f} ± {std_vals['neg_ddi']:.4f}")
print(f"Mean ± Std (Tolerability): {mean_vals['neg_tol']:.4f} ± {std_vals['neg_tol']:.4f}")

# Save summary report
summary = {
    'timestamp': timestamp,
    'n_eval_episodes': len(res_df),
    'mean_efficacy': float(mean_vals['efficacy']),
    'std_efficacy': float(std_vals['efficacy']),
    'mean_ddi': float(mean_vals['neg_ddi']),
    'std_ddi': float(std_vals['neg_ddi']),
    'mean_tolerability': float(mean_vals['neg_tol']),
    'std_tolerability': float(std_vals['neg_tol']),
    'model_path': model_path,
    'results_path': results_path,
    'plot_path': plot_path,
    'hyperparams_path': hyperparams_path,
    'model_based_learning': False,
    'pycddlib_patch': 'scipy_based'
}
summary_path = os.path.join(save_dir, f'summary_{timestamp}.json')
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)
print(f"💾 Saved summary report: {summary_path}")

print(f"\n🎉 Phase 7b Evaluation Complete! All artifacts saved to: {save_dir}")


NameError: name 'existing_models' is not defined

Analyze Results

In [17]:
# Phase 8 — Post-training analysis & publication-grade figures
# Save as phase8_analysis.py or paste in notebook cell and run.

import os
import glob
import json
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import ceil
import torch as th

# ---------- USER CONFIG ----------
SAVE_DIR = "/content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results"
RESULTS_PREFIX = "phase8_results"
EPISODES_PER_WEIGHT = 50          # episodes per weight
N_WEIGHTS = 40                    # number of weights to sample on simplex
TOP_K = 5                         # top/bottom episodes to save
RANDOM_SEED = 42
# ---------------------------------

np.random.seed(RANDOM_SEED)
th.manual_seed(RANDOM_SEED)

os.makedirs(SAVE_DIR, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_dir = os.path.join(SAVE_DIR, f"{RESULTS_PREFIX}_{timestamp}")
os.makedirs(results_dir, exist_ok=True)

print("Phase 8 results folder:", results_dir)

# ---------- helpers ----------
def find_latest_model_tar(save_dir):
    files = sorted(glob.glob(os.path.join(save_dir, "gpipd_model_*.tar")), key=os.path.getmtime)
    return files[-1] if files else None

def sample_weights_dirichlet(d, n, seed=None):
    rng = np.random.default_rng(seed)
    a = rng.gamma(1.0, 1.0, size=(n, d))
    a = a / a.sum(axis=1, keepdims=True)
    return a

def nondominated(points):
    """Return mask of nondominated points (higher is better)."""
    N = points.shape[0]
    mask = np.ones(N, dtype=bool)
    for i in range(N):
        if not mask[i]:
            continue
        # j dominates i if j>=i in all dims and > in at least one
        ge = np.all(points >= points[i], axis=1)
        gt = np.any(points > points[i], axis=1)
        dominated = np.any(ge & gt)
        if dominated:
            mask[i] = False
    return mask

# ---------- load / validate agent ----------
try:
    agent  # if already loaded in notebook
    print("Using existing 'agent' in workspace.")
except NameError:
    latest = find_latest_model_tar(SAVE_DIR)
    if latest is None:
        raise FileNotFoundError(f"No gpipd_model_*.tar found in {SAVE_DIR}. Run Phase 7 first.")
    print("Loading model:", latest)
    from morl_baselines.multi_policy.gpi_pd.gpi_pd import GPIPD
    # Build shell agent (must match architecture used in training)
    agent = GPIPD(env=train_env, learning_rate=3e-4, gamma=0.99, batch_size=256,
                  net_arch=[128,128], buffer_size=int(5e4), initial_epsilon=0.05, final_epsilon=0.05,
                  log=False, dyna=False)
    agent.load(path=latest)
    print("Model loaded.")

# quick checks
reward_dim = agent.reward_dim
action_space = agent.action_space
print(f"Agent reward_dim={reward_dim}, action_space={action_space}")

# ---------- sample weights ----------
W = sample_weights_dirichlet(reward_dim, N_WEIGHTS, seed=RANDOM_SEED)
print(f"Sampled {len(W)} weights on simplex.")

# ---------- evaluation loop ----------
all_entries = []  # list of dicts for JSON/CSV
for wi, w in enumerate(W):
    w_np = w.astype(np.float32)
    print(f"Weight {wi+1}/{len(W)}: {np.round(w_np,3)}")
    for ep in range(EPISODES_PER_WEIGHT):
        obs, _ = test_env.reset()
        done = False
        total = np.zeros(reward_dim, dtype=np.float32)
        actions = []
        info = {}
        while not done:
            # Use agent.eval(obs, w) for action selection
            a = agent.eval(obs, w_np)
            obs, r, done, trunc, info = test_env.step(int(a))
            total += r
            actions.append(int(a))
        entry = {
            "weight_index": int(wi),
            "weight": w_np.tolist(),
            "episode_index": int(ep),
            "total_rewards": total.tolist(),
            "hadm_id": info.get("hadm_id", None),
            "actions": actions
        }
        all_entries.append(entry)

# ---------- save raw JSON + flattened CSV ----------
raw_json = os.path.join(results_dir, "phase8_raw_results.json")
with open(raw_json, "w") as f:
    json.dump(all_entries, f)
print("Saved raw JSON:", raw_json)

# flatten to dataframe
rows = []
for e in all_entries:
    row = {
        "weight_index": e["weight_index"],
        "weight": e["weight"],
        "episode_index": e["episode_index"],
        "hadm_id": e["hadm_id"],
    }
    for i, val in enumerate(e["total_rewards"]):
        row[f"obj{i}"] = val
    row["actions"] = e["actions"]
    rows.append(row)

df = pd.DataFrame(rows)
flat_csv = os.path.join(results_dir, "phase8_flat_results.csv")
df.to_csv(flat_csv, index=False)
print("Saved flattened CSV:", flat_csv)

# ---------- Pareto frontier by mean reward per weight ----------
grouped = df.groupby("weight_index")[[f"obj{i}" for i in range(reward_dim)]].mean().reset_index()
points = grouped[[f"obj{i}" for i in range(reward_dim)]].values
nd_mask = nondominated(points)
pareto_df = grouped.loc[nd_mask].copy()
pareto_df["weight"] = grouped.loc[nd_mask, "weight_index"].apply(lambda idx: W[int(idx)].tolist())
pareto_csv = os.path.join(results_dir, "pareto_front_mean_by_weight.csv")
pareto_df.to_csv(pareto_csv, index=False)
print("Saved Pareto CSV:", pareto_csv)

# ---------- Pareto plot ----------
plt.figure(figsize=(8,6))
if reward_dim == 2:
    plt.scatter(points[:,0], points[:,1], alpha=0.5)
    plt.scatter(pareto_df[f"obj0"], pareto_df[f"obj1"], color="red", s=80, label="Pareto")
    plt.xlabel("obj0"); plt.ylabel("obj1")
elif reward_dim == 3:
    sc = plt.scatter(points[:,0], points[:,1], c=points[:,2], cmap="viridis", alpha=0.7)
    plt.scatter(pareto_df[f"obj0"], pareto_df[f"obj1"], facecolors='none', edgecolors='r', s=120, linewidths=1.5)
    plt.xlabel("obj0"); plt.ylabel("obj1"); plt.colorbar(sc, label="obj2")
else:
    plt.scatter(points[:,0], points[:,1], alpha=0.6)
    plt.scatter(pareto_df[f"obj0"], pareto_df[f"obj1"], color="red", s=80)
    plt.xlabel("obj0"); plt.ylabel("obj1")
plt.title("Pareto front (mean rewards per weight)")
plt.grid(alpha=0.2)
plt.tight_layout()
pareto_plot = os.path.join(results_dir, "pareto_front_plot.png")
plt.savefig(pareto_plot, dpi=300)
plt.close()
print("Saved Pareto plot:", pareto_plot)

# ---------- action frequency heatmap ----------
# list all unique actions seen
unique_actions = sorted({a for seq in df["actions"] for a in seq})
n_actions = len(unique_actions)
freq = np.zeros((len(W), n_actions), dtype=int)
for _, r in df.iterrows():
    wi = int(r["weight_index"])
    for a in r["actions"]:
        freq[wi, unique_actions.index(a)] += 1

# normalize rows
row_sums = freq.sum(axis=1, keepdims=True)
row_sums[row_sums==0] = 1
freq_norm = freq / row_sums

plt.figure(figsize=(10, max(4, len(W)*0.12)))
plt.imshow(freq_norm, aspect='auto', interpolation='nearest')
plt.colorbar(label="fraction")
plt.yticks(np.arange(len(W)), [f"w{idx}" for idx in range(len(W))])
plt.xticks(np.arange(n_actions), unique_actions, rotation=90)
plt.xlabel("action"); plt.ylabel("weight index")
plt.title("Action frequency (weights x actions)")
plt.tight_layout()
heatmap_path = os.path.join(results_dir, "action_frequency_heatmap.png")
plt.savefig(heatmap_path, dpi=300)
plt.close()
print("Saved heatmap:", heatmap_path)
pd.DataFrame(freq, columns=[f"action_{a}" for a in unique_actions]).to_csv(os.path.join(results_dir, "action_freq_counts.csv"), index=False)

# ---------- radar/tradeoff plots for representative weights ----------
# pick representative indices: extremes + some pareto ones
rep_idxs = [0, len(W)-1] + list(map(int, grouped.loc[nd_mask, "weight_index"].tolist()[:4]))
rep_idxs = [i for i in dict.fromkeys(rep_idxs) if 0 <= i < len(W)]
labels = [f"obj{i}" for i in range(reward_dim)]
angles = np.linspace(0, 2*np.pi, reward_dim, endpoint=False).tolist()
angles += angles[:1]

n_cols = 2
n_rows = ceil(len(rep_idxs)/n_cols)
plt.figure(figsize=(6*n_cols, 4*n_rows))
for k, idx in enumerate(rep_idxs):
    vals = grouped[grouped.weight_index==idx][[f"obj{i}" for i in range(reward_dim)]].values.flatten().tolist()
    vals += vals[:1]
    ax = plt.subplot(n_rows, n_cols, k+1, polar=True)
    ax.plot(angles, vals, linewidth=2)
    ax.fill(angles, vals, alpha=0.2)
    ax.set_thetagrids(np.degrees(angles[:-1]), labels)
    ax.set_title(f"weight idx {idx}")
plt.tight_layout()
radar_path = os.path.join(results_dir, "tradeoff_radar_plots.png")
plt.savefig(radar_path, dpi=300)
plt.close()
print("Saved radar plots:", radar_path)

# ---------- reward correlation matrix ----------
objs = [f"obj{i}" for i in range(reward_dim)]
corr = df[objs].corr()
plt.figure(figsize=(6,5))
plt.imshow(corr, vmin=-1, vmax=1, cmap="coolwarm")
plt.colorbar()
plt.xticks(range(len(objs)), objs)
plt.yticks(range(len(objs)), objs)
for (i,j), val in np.ndenumerate(corr.values):
    plt.text(j, i, f"{val:.2f}", ha='center', va='center', color='white' if abs(val)>0.5 else 'black')
corr_path = os.path.join(results_dir, "reward_correlation_matrix.png")
plt.savefig(corr_path, dpi=300)
plt.close()
corr.to_csv(os.path.join(results_dir, "reward_correlation_matrix.csv"))
print("Saved correlation matrix:", corr_path)

# ---------- top/bottom episodes ----------
for i in range(reward_dim):
    col = f"obj{i}"
    sorted_df = df.sort_values(col, ascending=False)
    top = sorted_df.head(TOP_K)[["weight_index","episode_index","hadm_id",col,"actions"]]
    bottom = sorted_df.tail(TOP_K)[["weight_index","episode_index","hadm_id",col,"actions"]]
    top.to_csv(os.path.join(results_dir, f"top_{i}_episodes.csv"), index=False)
    bottom.to_csv(os.path.join(results_dir, f"bottom_{i}_episodes.csv"), index=False)
print("Saved top/bottom episode tables.")

# ---------- minimal JSON summary ----------
summary = {
    "timestamp": timestamp,
    "n_weights": len(W),
    "episodes_per_weight": EPISODES_PER_WEIGHT,
    "pareto_count": int(nd_mask.sum()),
    "raw_json": raw_json,
    "flat_csv": flat_csv,
    "pareto_csv": pareto_csv,
    "pareto_plot": pareto_plot,
    "heatmap": heatmap_path,
    "radar": radar_path,
    "corr_matrix": corr_path,
}
with open(os.path.join(results_dir, "phase8_summary.json"), "w") as f:
    json.dump(summary, f, indent=2)

print("Phase 8 complete. All artifacts saved to:", results_dir)


Phase 8 results folder: /content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/phase8_results_20251118_050009
Using existing 'agent' in workspace.
Agent reward_dim=3, action_space=Discrete(2)
Sampled 40 weights on simplex.
Weight 1/40: [0.337 0.328 0.335]
Weight 2/40: [0.154 0.048 0.799]
Weight 3/40: [0.306 0.677 0.017]
Weight 4/40: [0.474 0.032 0.494]
Weight 5/40: [0.517 0.115 0.368]
Weight 6/40: [0.274 0.163 0.562]
Weight 7/40: [0.352 0.161 0.487]
Weight 8/40: [0.068 0.559 0.373]
Weight 9/40: [0.43  0.273 0.297]
Weight 10/40: [0.082 0.191 0.727]
Weight 11/40: [0.165 0.535 0.3  ]
Weight 12/40: [0.177 0.344 0.479]
Weight 13/40: [0.223 0.208 0.569]
Weight 14/40: [0.098 0.441 0.46 ]
Weight 15/40: [0.473 0.032 0.495]
Weight 16/40: [0.491 0.407 0.102]
Weight 17/40: [0.382 0.197 0.421]
Weight 18/40: [0.014 0.111 0.875]
Weight 19/40: [0.825 0.116 0.059]
Weight 20/40: [0.448 0.407 0.146]
Weight 21/40: [0.273 0.347 0.38 ]
Weight 22/40: [0.586 0.232 0.182]
Weight 23/40: [0.621 0.3

In [18]:
# Phase 8 — Generate PDF report from Phase8 artifacts
import os
import json
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from PIL import Image

# ---------------- CONFIG ----------------
# set results_dir to the same folder you used in Phase 8
# If you run this immediately after Phase 8, results_dir exists in the session.
try:
    results_dir  # noqa
except NameError:
    # fallback: change this path to the timestamped folder created by Phase 8
    results_dir = "/content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/phase8_results_YYYYMMDD_HHMMSS"

pdf_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
pdf_path = os.path.join(results_dir, f"phase8_report_{pdf_timestamp}.pdf")
meta_json = os.path.join(results_dir, "phase8_summary.json")
flat_csv = os.path.join(results_dir, "phase8_flat_results.csv")

# Candidate figure files (names used by Phase 8)
candidates = {
    "pareto_plot": os.path.join(results_dir, "pareto_front_plot.png"),
    "heatmap": os.path.join(results_dir, "action_frequency_heatmap.png"),
    "radar": os.path.join(results_dir, "tradeoff_radar_plots.png"),
    "corr": os.path.join(results_dir, "reward_correlation_matrix.png"),
    # Add any additional images you want to include if they exist
    "pareto_front_mean_by_weight": os.path.join(results_dir, "pareto_front_mean_by_weight.csv"),
}

# Load summary if present
summary = {}
if os.path.exists(meta_json):
    try:
        with open(meta_json, "r") as f:
            summary = json.load(f)
    except Exception:
        summary = {}

# Helper to safely add an image page
def add_image_page(pp: PdfPages, image_path: str, title: str = "", caption: str = ""):
    fig, ax = plt.subplots(figsize=(11.69, 8.27))  # A4 landscape
    ax.axis("off")
    try:
        img = Image.open(image_path)
        ax.imshow(img)
    except Exception as e:
        ax.text(0.5, 0.5, f"Could not load image:\n{image_path}\n\n{e}", ha="center", va="center", wrap=True)
    # Title
    if title:
        fig.suptitle(title, fontsize=14, fontweight="bold", y=0.98)
    # Caption in footer
    if caption:
        plt.figtext(0.02, 0.02, caption, wrap=True, ha="left", fontsize=9)
    pp.savefig(fig)
    plt.close(fig)

# ---------------- Build PDF ----------------
with PdfPages(pdf_path) as pp:
    # Cover page
    fig, ax = plt.subplots(figsize=(11.69, 8.27))
    ax.axis("off")
    title = "Phase 8 — Post-Training Analysis Report"
    ax.text(0.5, 0.78, title, ha="center", va="center", fontsize=22, fontweight="bold")
    sub = f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    ax.text(0.5, 0.74, sub, ha="center", va="center", fontsize=10)

    # Summary block: try to extract basic stats
    n_weights = summary.get("n_weights", "N/A")
    episodes_per_weight = summary.get("episodes_per_weight", summary.get("n_eval_episodes_per_weight", "N/A"))
    pareto_count = summary.get("pareto_count", "N/A")
    mean_rewards = summary.get("mean_rewards", {})
    std_rewards = summary.get("std_rewards", {})

    left_block = (
        f"Results folder: {os.path.basename(results_dir)}\n\n"
        f"Weights sampled: {n_weights}\n"
        f"Episodes/weight: {episodes_per_weight}\n"
        f"Pareto points found (by mean): {pareto_count}\n"
    )
    ax.text(0.05, 0.55, left_block, fontsize=10, ha="left", va="top", family="monospace")

    # Add mean/std table if available
    ms_lines = []
    if mean_rewards:
        ms_lines.append("Mean ± Std (per objective):")
        for k, v in mean_rewards.items():
            stdv = std_rewards.get(k, 0.0)
            ms_lines.append(f"  {k}: {v:.4f} ± {stdv:.4f}")
    else:
        ms_lines.append("Mean/std not found in summary.")

    ax.text(0.05, 0.35, "\n".join(ms_lines), fontsize=9, ha="left", va="top", family="monospace")

    # Auto-generated interpretation (short)
    interp = (
        "Interpretation (auto-generated):\n"
        "• The Pareto plot shows trade-offs between objectives across sampled weight vectors.\n"
        "• The action-frequency heatmap highlights which actions (drugs) are preferred under different weights.\n"
        "• Radar plots summarize per-weight tradeoffs; correlation matrix shows objective correlations.\n\n"
        "Limitations:\n"
        "• Evaluation uses agent.eval() with per-weight greedy selection — may not capture stochasticity.\n"
        "• Clinical interpretation requires mapping actions → drug names and domain expert review.\n"
    )
    ax.text(0.05, 0.06, interp, fontsize=9, ha="left", va="bottom")
    pp.savefig(fig)
    plt.close(fig)

    # Add available figures with captions
    if os.path.exists(candidates["pareto_plot"]):
        add_image_page(pp, candidates["pareto_plot"], title="Pareto Scatter (mean per weight)",
                       caption="Pareto scatter: mean reward vector for each sampled weight. Red markers indicate estimated Pareto frontier.")
    else:
        add_image_page(pp, candidates["pareto_plot"], title="Pareto Scatter", caption="Image not found; skipped.")

    if os.path.exists(candidates["heatmap"]):
        add_image_page(pp, candidates["heatmap"], title="Action Frequency Heatmap",
                       caption="Rows = weight vectors, Columns = action indices. Values = fraction of picks across episodes.")
    if os.path.exists(candidates["radar"]):
        add_image_page(pp, candidates["radar"], title="Tradeoff Radar Plots",
                       caption="Radar plots of mean objective values for representative weight vectors.")
    if os.path.exists(candidates["corr"]):
        add_image_page(pp, candidates["corr"], title="Reward Correlation Matrix",
                       caption="Pearson correlation matrix between objective returns across all evaluated episodes.")

    # Add a raw CSV preview page (first few rows) if exists
    if os.path.exists(flat_csv := os.path.join(results_dir, "phase8_flat_results.csv")):
        try:
            df_preview = pd.read_csv(flat_csv).head(20)
            fig, ax = plt.subplots(figsize=(11.69, 8.27))
            ax.axis("off")
            ax.table(cellText=df_preview.values, colLabels=df_preview.columns, loc="center", cellLoc="center")
            fig.suptitle("Flat results preview (first 20 rows)", fontsize=12)
            pp.savefig(fig)
            plt.close(fig)
        except Exception:
            pass

    # Final page: recommended next steps
    fig, ax = plt.subplots(figsize=(11.69, 8.27))
    ax.axis("off")
    next_steps = [
        "Recommended next steps:",
        "1) Map action indices → drug names and produce a clinical interpretation table.",
        "2) Run targeted weight sweeps around Pareto regions for higher resolution.",
        "3) Build a Streamlit demo to interactively toggle weights and visualize policy actions.",
        "4) Prepare a write-up or manuscript with methods, experiments, and clinical validation plan."
    ]
    ax.text(0.05, 0.9, "Next steps (suggested)", fontsize=14, fontweight="bold")
    ax.text(0.05, 0.6, "\n".join(next_steps), fontsize=11, va="top")
    ax.text(0.05, 0.05, f"Report generated from folder: {os.path.basename(results_dir)}", fontsize=8, va="bottom")
    pp.savefig(fig)
    plt.close(fig)

print("✅ PDF report generated at:", pdf_path)


✅ PDF report generated at: /content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/phase8_results_20251118_050009/phase8_report_20251118_050115.pdf


In [20]:
# 1_update_summary_mean_std.py
import os, json, pandas as pd

RESULTS_DIR = "/content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/phase8_results_20251118_050009"
SUMMARY_PATH = os.path.join(RESULTS_DIR, "phase8_summary.json")
FLAT_CSV = os.path.join(RESULTS_DIR, "phase8_flat_results.csv")

# Load flat results
df = pd.read_csv(FLAT_CSV)
# Detect objective columns automatically (obj0, obj1, ...)
obj_cols = [c for c in df.columns if c.startswith("obj")]
if len(obj_cols) == 0:
    raise RuntimeError("No objective columns (obj0...) found in flat CSV: " + FLAT_CSV)

# Compute mean/std
mean_rewards = df[obj_cols].mean().to_dict()
std_rewards  = df[obj_cols].std().to_dict()

# Load existing summary (or create)
if os.path.exists(SUMMARY_PATH):
    with open(SUMMARY_PATH, "r") as f:
        summary = json.load(f)
else:
    summary = {}

# Update
summary["mean_rewards"] = {k: float(v) for k,v in mean_rewards.items()}
summary["std_rewards"]  = {k: float(v) for k,v in std_rewards.items()}

# Save back
with open(SUMMARY_PATH, "w") as f:
    json.dump(summary, f, indent=2)

print("Updated summary JSON with mean/std at:", SUMMARY_PATH)
print("Mean rewards:", summary["mean_rewards"])
print("Std rewards:", summary["std_rewards"])


Updated summary JSON with mean/std at: /content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/phase8_results_20251118_050009/phase8_summary.json
Mean rewards: {'obj0': 0.12897999889403583, 'obj1': -0.02736000108718872, 'obj2': -2.630034459605813}
Std rewards: {'obj0': 0.22932434108126395, 'obj1': 0.13769572238106279, 'obj2': 2.525901678121713}


In [31]:
# compute_policy_stats_and_figures.py
import os, json, ast
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

RESULTS_DIR = "/content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/phase8_results_20251118_050009"
FLAT_CSV = os.path.join(RESULTS_DIR, "phase8_flat_results.csv")
SUMMARY_JSON = os.path.join(RESULTS_DIR, "phase8_summary.json")

# load
df = pd.read_csv(FLAT_CSV)
# determine objective columns automatically
obj_cols = [c for c in df.columns if c.startswith("obj")]
reward_dim = len(obj_cols)
print("Detected objective columns:", obj_cols)

# parse actions column (if stored as list-like strings)
def parse_actions(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception:
            # fallback: simple split
            return [int(v) for v in x.strip("[]").split(",") if v!='']
    if isinstance(x, (list, tuple, np.ndarray)):
        return list(x)
    return []

df['actions_parsed'] = df['actions'].apply(parse_actions)

# -- Frequency: how often each action/policy is taken across ALL episodes --
from collections import Counter
counter = Counter()
for seq in df['actions_parsed']:
    counter.update(seq)
action_indices = sorted(counter.keys())
freq_df = pd.DataFrame({
    'action_index': action_indices,
    'count': [counter[i] for i in action_indices]
})
freq_df['fraction_of_total_action_calls'] = freq_df['count'] / freq_df['count'].sum()

# -- Per-action episode-level presence & mean per-objective totals --
# join per-episode totals to actions: each row in df is an episode's totals
exploded_rows = []
for _, row in df.iterrows():
    acts = row['actions_parsed']
    for a in acts:
        d = {c: row[c] for c in obj_cols}
        d.update({
            'weight_index': int(row['weight_index']),
            'episode_index': int(row['episode_index']),
            'action': int(a)
        })
        exploded_rows.append(d)
expl = pd.DataFrame(exploded_rows)

# episodes containing each action (unique episode counts)
presence_counts = expl.groupby('action').agg({'episode_index':'nunique'}).rename(columns={'episode_index':'n_episodes_with_action'}).reset_index()

# average episode totals for episodes where action appears (coarse proxy)
agg = expl.groupby('action')[obj_cols].mean().reset_index()
agg = agg.merge(presence_counts, on='action', how='left')
agg = agg.merge(freq_df.rename(columns={'action_index':'action'}), left_on='action', right_on='action', how='left')

# Save tables
freq_path = os.path.join(RESULTS_DIR, "action_frequency_by_policy.csv")
agg_path  = os.path.join(RESULTS_DIR, "per_policy_outcome_stats.csv")
freq_df.to_csv(freq_path, index=False)
agg.to_csv(agg_path, index=False)

print("Saved:", freq_path)
print("Saved:", agg_path)
print(agg.head())

# -- Figures --
# 1) Bar chart: frequency of each action (policy)
plt.figure(figsize=(6,4))
plt.bar(freq_df['action_index'].astype(str), freq_df['count'])
plt.xlabel("Action (policy index)")
plt.ylabel("Total action calls")
plt.title("Action (policy) call counts across all evaluated episodes")
plt.tight_layout()
freq_plot = os.path.join(RESULTS_DIR, "action_frequency_barplot.png")
plt.savefig(freq_plot, dpi=300)
plt.close()
print("Saved:", freq_plot)

# 2) Radar-like spider for per-policy mean objectives (simple line plot)
plt.figure(figsize=(6,4))
for _, r in agg.iterrows():
    vals = [r[c] for c in obj_cols]
    plt.plot(range(len(vals)), vals, marker='o', label=f"policy {int(r['action'])}")
plt.xticks(range(len(obj_cols)), obj_cols)
plt.xlabel("Objective")
plt.ylabel("Mean episode total (for episodes where policy appears)")
plt.title("Per-policy mean objective totals (episodes where policy present)")
plt.legend()
plt.tight_layout()
radar_plot = os.path.join(RESULTS_DIR, "per_policy_mean_objectives.png")
plt.savefig(radar_plot, dpi=300)
plt.close()
print("Saved:", radar_plot)

# 3) Optional: save small summary JSON with top/bottom policies by tolerability
summary = {}
summary['top_policy_by_tolerability'] = agg.sort_values('obj2', ascending=False).head(1)['action'].tolist()
summary['bottom_policy_by_tolerability'] = agg.sort_values('obj2', ascending=True).head(1)['action'].tolist()
with open(SUMMARY_JSON, 'r') as f:
    s = json.load(f)
s.update(summary)
with open(SUMMARY_JSON, 'w') as f:
    json.dump(s, f, indent=2)
print("Appended quick policy rankings to summary JSON.")


Detected objective columns: ['obj0', 'obj1', 'obj2']
Saved: /content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/phase8_results_20251118_050009/action_frequency_by_policy.csv
Saved: /content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/phase8_results_20251118_050009/per_policy_outcome_stats.csv
   action      obj0      obj1      obj2  n_episodes_with_action  count  \
0       0  0.128877 -0.028047 -2.607255                      50   1951   
1       1  0.133061  0.000000 -3.537041                      42     49   

   fraction_of_total_action_calls  
0                          0.9755  
1                          0.0245  
Saved: /content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/phase8_results_20251118_050009/action_frequency_barplot.png
Saved: /content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/phase8_results_20251118_050009/per_policy_mean_objectives.png
Appended quick policy rankings to summary JSON.


In [32]:
MANUSCRIPT = "/content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/manuscript.md"
CLINICAL_MD = "/content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/clinical_interp.md"

# write clinical_md using the template above and the computed agg table
# (run after compute_policy_stats_and_figures.py so CSVs exist)
import pandas as pd, os
RESULTS_DIR = "/content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/phase8_results_20251118_050009"
agg = pd.read_csv(os.path.join(RESULTS_DIR, "per_policy_outcome_stats.csv"))
freq = pd.read_csv(os.path.join(RESULTS_DIR, "action_frequency_by_policy.csv"))

p0 = agg[agg['action']==0].iloc[0]
p1 = agg[agg['action']==1].iloc[0]

clinical_text = f'''
## Clinical interpretation

**Environment & decision abstraction.**
The agent operates over a high-level treatment-policy selection environment (PolypharmacyEnv). The action space is binary (Discrete(2)) and corresponds to two treatment policies rather than individual drug selections. We therefore interpret actions as **policy-level decisions**:

- **Policy 0** — Conservative treatment policy (lower predicted tolerability burden).
- **Policy 1** — Aggressive treatment policy (higher predicted tolerability burden).

**How policies behave in evaluation.**
Policy call counts (total calls across all evaluated episodes):
- Policy 0 total calls: {int(freq[freq['action_index']==0]['count'].values[0])}
- Policy 1 total calls: {int(freq[freq['action_index']==1]['count'].values[0])}

Per-policy mean rewards (computed only over episodes where that policy was present):
- Policy 0 mean rewards ({" , ".join(obj_cols)}): {', '.join([f'{c}: {p0[c]:.4f}' for c in obj_cols])}
- Policy 1 mean rewards ({" , ".join(obj_cols)}): {', '.join([f'{c}: {p1[c]:.4f}' for c in obj_cols])}

> Interpretation: policy 0 yields relatively better tolerability (less negative `neg_tol`) than policy 1, at similar efficacy levels. Policy 1 tends to produce a more negative tolerability score, indicating higher adverse effect burden in episodes where it is present.

**Clinical implications and caveats.**
1. These results do **not** directly translate to drug-level prescriptions. Instead, they indicate the agent’s preference between two coarse strategies (conservative vs aggressive).
2. The tolerability outcome (`neg_tol`) is a proxy derived from dataset features and the environment’s reward function; it should be validated with clinicians before any actionable inference.
3. The per-policy mean outcomes are associative: an episode where a policy appears and shows worse tolerability is not evidence that the policy causes worse outcomes without causal analysis or controlled experiments.

'''
# append
with open(CLINICAL_MD, 'w') as f:
    f.write(clinical_text)

with open(MANUSCRIPT, 'a') as f:
    f.write("\n\n")
    f.write(clinical_text)

print("Wrote clinical interpretation to", CLINICAL_MD, "and appended to", MANUSCRIPT)


Wrote clinical interpretation to /content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/clinical_interp.md and appended to /content/drive/MyDrive/ML Patent/polypharmacy_project/phase7_results/manuscript.md


clean model

In [41]:
import torch

ckpt = torch.load("gpipd_model_clean.pt", map_location="cpu")

print("=== Q-NET 0 STATE_DICT SHAPES ===")
sd = ckpt["q_nets"][0]
for k,v in sd.items():
    print(k, v.shape)


=== Q-NET 0 STATE_DICT SHAPES ===
weights_features.0.weight torch.Size([128, 3])
weights_features.0.bias torch.Size([128])
state_features.0.weight torch.Size([128, 4])
state_features.0.bias torch.Size([128])
net.0.weight torch.Size([128, 128])
net.0.bias torch.Size([128])
net.2.weight torch.Size([128])
net.2.bias torch.Size([128])
net.4.weight torch.Size([6, 128])
net.4.bias torch.Size([6])


In [42]:
for i, sd in enumerate(ckpt["q_nets"]):
    print(f"\n=== Q-NET {i} ===")
    for k,v in sd.items():
        print(k, tuple(v.shape))



=== Q-NET 0 ===
weights_features.0.weight (128, 3)
weights_features.0.bias (128,)
state_features.0.weight (128, 4)
state_features.0.bias (128,)
net.0.weight (128, 128)
net.0.bias (128,)
net.2.weight (128,)
net.2.bias (128,)
net.4.weight (6, 128)
net.4.bias (6,)

=== Q-NET 1 ===
weights_features.0.weight (128, 3)
weights_features.0.bias (128,)
state_features.0.weight (128, 4)
state_features.0.bias (128,)
net.0.weight (128, 128)
net.0.bias (128,)
net.2.weight (128,)
net.2.bias (128,)
net.4.weight (6, 128)
net.4.bias (6,)
